In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from matplotlib import pyplot as plt
import glob
import numpy as np
from skimage import io
#import random
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier





def testClassifier(classifier):
    fitClassifiers.append(classifier)
    scores.append(classifier.score(XTest, yTest))
    for j in range(numClasses):
        classScores[j].append(classifier.score(np.array([io.imread(file)[0,:,0:2].flatten()/255 for file in classPathsTest[j]]),[j]*len(classPathsTest[j])))
        for file in classPathsTest[j]:
            x = classifier.predict((io.imread(file)[0,:,0:2].flatten()/255).reshape(1, -1))[0]
            if not x==j:
                wrongFiles[nClassifier].append((file,j,x))
            confusionMatrix[nClassifier][j][x] += 1
   
    
imagePaths = glob.glob('./pieces_dataV3/spectrum/*.png')
#for index in range(len(imagePaths)):
#    print(index, imagePaths[index])
classes = ["A", "B", "C", "D", "E", "F", "G", "H", "J", "K", "L", "M", "N", "P", "Q", "R", "S"]
groupClasses = [0,
               0,
               1,
               1,
               1,
               1,
               1,
               1,
               1,
               1,
               2,
               2,
               3,
               3,
               3,
               4,
               5]
groupNames = ["Campaniana","Terra sigillata","Cuina","Àmfora","Construcció romana","Construcció modern"]
classifierNames = ["Gaussian Process","Gaussian Naive-Bayes","Decision Tree","Random Forest","SVC","K-Neighbors"]
classesStart = [723, 813, 902, 992,1082, 1172, 1262, 1352, 1442, 1526, 1616, 1706, 1796, 1886, 1977, 2067, 2157, len(imagePaths)]
numClasses = len(classes)
numClassifiers = len(classifierNames)
wrongFiles = [[],[],[],[],[],[]]
meanScores = [0,0,0,0,0,0]
meanClassScore = [0]*numClasses
for j in range(numClasses):
    meanClassScore[j] = [0,0,0,0,0,0]
confusionMatrix = [0]*numClassifiers
for j in range(numClassifiers):
    confusionMatrix[j] = [0]*numClasses
    for k in range(numClasses):
        confusionMatrix[j][k] = [0]*numClasses
classPaths = [0]*numClasses
for i in range(numClasses):
    classPaths[i] = imagePaths[classesStart[i]:classesStart[i+1]]


kernel = 1.0 * RBF(1.0)
folds = 10
matrices = []

for i in range(folds):
    print("fold "+str(i+1)+" of "+str(folds))
    scores = []
    nClassifier = 0
    classScores = [0]*numClasses
    for j in range(numClasses):
        classScores[j] = []
    
    classPathsTrain = [0]*numClasses
    classPathsTest = [0]*numClasses
    matrices = []
    #numberOfWrongFiles = []
    fitClassifiers = []
    paths = []
    y = []
    pathsTest = []
    yTest = []
    
    for j in range(numClasses):
        length = len(classPaths[j])
        testBegin = int(i*length/folds)
        testEnd = int((i+1)*length/folds)
        #print(testBegin, testEnd, len(iber))
        classTest = classPaths[j][testBegin:testEnd]
        classTrain = classPaths[j][:testBegin]
        classTrain.extend(classPaths[j][testEnd:])
        classPathsTrain[j] = classTrain
        classPathsTest[j] = classTest
              
        paths.extend(classTrain)
        y.extend([j]*len(classTrain))
        pathsTest.extend(classTest)
        yTest.extend([j]*len(classTest))
    
    X = np.array([io.imread(file)[0,:,0:2].flatten()/255 for file in paths])
    XTest = np.array([io.imread(file)[0,:,0:2].flatten()/255 for file in pathsTest])
    
    classifier = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(X, y)
    testClassifier(classifier)
    nClassifier = 1
    classifier = GaussianNB().fit(X, y)
    testClassifier(classifier)
    nClassifier = 2
    classifier = DecisionTreeClassifier(random_state=0).fit(X, y)
    testClassifier(classifier)
    nClassifier = 3
    classifier = RandomForestClassifier(random_state=0).fit(X, y)
    testClassifier(classifier)
    nClassifier = 4
    classifier = make_pipeline(StandardScaler(), SVC(gamma='auto')).fit(X, y)
    testClassifier(classifier)
    nClassifier = 5
    classifier = KNeighborsClassifier().fit(X, y)
    testClassifier(classifier)
    for k in range(6):
        '''print(meanScores)
        print(scores)
        print(k)
        print(len(meanScores))
        print(len(scores))
        
        print(scores[k])'''
        meanScores[k]+=scores[k]
        for j in range(numClasses):
            meanClassScore[j][k] += classScores[j][k]
    '''print("________________________")
    print(meanScores)
    print("________________________")
    print("------------------------")
    print(meanClassScore)
    print("------------------------")
    print("++++++++++++++++++++++++")
    print(confusionMatrix)
    print("++++++++++++++++++++++++")
    print(wrongFiles)'''


for k in range(6):
    for j in range(numClasses):
        meanClassScore[j][k] /= folds
    meanScores[k] /= folds

print("\n\nResults")
for i in range(6):
    print(classifierNames[i]+":"+"\n\tAccuracy: {:.2f}".format(meanScores[i]))
    for j in range(numClasses):
        print("\tAccuracy for class"+classes[j]+": {:.2f}".format(meanClassScore[j][i]))
    print()
print("\n\nWrongly predicted files:")
for i in range(6):
    file= open(classifierNames[i]+"17_classes_wrongFiles.txt","w+")
    print(classifierNames[i]+":")
    for j in range(len(wrongFiles[i])):
        file.write(str(wrongFiles[i][j][0])+" correct:"+str(wrongFiles[i][j][1])+" predicted:"+str(wrongFiles[i][j][2])+"\n")
        print(wrongFiles[i][j])
    print()
    file.close()
    
print(confusionMatrix)
numClassifiers = len(classifierNames)
numRows = 2
fig, axes = plt.subplots(nrows=numRows, ncols=numClassifiers//numRows, figsize=(40,20),sharey=False)
for i in range(numClassifiers):
    ax = axes.flatten()[i]
    ax.set_title(classifierNames[i]+"\nAccuracy: {:.2f}".format(meanScores[i]), pad=20,fontdict={"weight":"bold"})
    arr = np.array([np.array(xi) for xi in confusionMatrix[i]])
    newArray = np.zeros(arr.shape, dtype=np.float32)
    sumArr = np.sum(arr, axis=1)
    for row in range(arr.shape[0]):
        newArray[row] = np.divide(arr[row],sumArr[row], casting='unsafe')
    ax.imshow(newArray, cmap ='Blues')
    ax.set_xticks(np.arange(numClasses))
    ax.set_yticks(np.arange(numClasses))
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)
    for j in range(numClasses):
        for k in range(numClasses):
            
            #text = ax.text(k, j, "%.0f" % confusionMatrix[i][j][k],ha="center", va="center", color="black")
            text = ax.text(k, j, "%.0f" % arr[j][k],ha="center", va="center", color="black")

plt.tight_layout()
plt.show()


fold 1 of 10
fold 2 of 10


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from matplotlib import pyplot as plt
import glob
import numpy as np
from skimage import io
#import random
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier





def testClassifier(classifier):
    fitClassifiers.append(classifier)
    scores.append(classifier.score(XTest, yTest))
    for j in range(numClasses):
        '''print(classScores)
        print(len(classScores))
        print(groupClasses[j])
        print(classScores[groupClasses[j]])
        print(j)'''
        classScores[groupClasses[j]].append(classifier.score(np.array([io.imread(file)[0,:,0:2].flatten()/255 for file in classPathsTest[j]]),[groupClasses[j]]*len(classPathsTest[j])))
        for file in classPathsTest[j]:
            x = classifier.predict((io.imread(file)[0,:,0:2].flatten()/255).reshape(1, -1))[0]
            if not x==j:
                wrongFiles[nClassifier].append((file,j,x))
            confusionMatrix[nClassifier][groupClasses[j]][x] += 1
   
    
imagePaths = glob.glob('./pieces_dataV3/spectrum/*.png')
#for index in range(len(imagePaths)):
#    print(index, imagePaths[index])
classes = ["A", "B", "C", "D", "E", "F", "G", "H", "J", "K", "L", "M", "N", "P", "Q", "R", "S"]
groupClasses = [0,
               0,
               1,
               1,
               1,
               1,
               1,
               1,
               1,
               1,
               2,
               2,
               3,
               3,
               3,
               4,
               5]
groupNames = ["Campaniana","Terra sigillata","Cuina","Àmfora","Construcció romana","Construcció modern"]
numGroups = len(groupNames)
classifierNames = ["Gaussian Process","Gaussian Naive-Bayes","Decision Tree","Random Forest","SVC","K-Neighbors"]
classesStart = [723, 813, 902, 992,1082, 1172, 1262, 1352, 1442, 1526, 1616, 1706, 1796, 1886, 1977, 2067, 2157, len(imagePaths)]
numClasses = len(classes)
numClassifiers = len(classifierNames)
wrongFiles = [[],[],[],[],[],[]]
meanScores = [0,0,0,0,0,0]
meanClassScore = [0]*numClasses
for j in range(numGroups):
    meanClassScore[j] = [0,0,0,0,0,0]
confusionMatrix = [0]*numClassifiers
for j in range(numClassifiers):
    confusionMatrix[j] = [0]*numGroups
    for k in range(numGroups):
        confusionMatrix[j][k] = [0]*numGroups
classPaths = [0]*numClasses
for i in range(numClasses):
    classPaths[i] = imagePaths[classesStart[i]:classesStart[i+1]]


kernel = 1.0 * RBF(1.0)
folds = 10
matrices = []

for i in range(folds):
    print("fold "+str(i+1)+" of "+str(folds))
    scores = []
    nClassifier = 0
    classScores = [0]*numGroups
    for j in range(numGroups):
        classScores[j] = []
    
    classPathsTrain = [0]*numClasses
    classPathsTest = [0]*numClasses
    matrices = []
    #numberOfWrongFiles = []
    fitClassifiers = []
    paths = []
    y = []
    pathsTest = []
    yTest = []
    
    for j in range(numClasses):
        length = len(classPaths[j])
        testBegin = int(i*length/folds)
        testEnd = int((i+1)*length/folds)
        #print(testBegin, testEnd, len(iber))
        classTest = classPaths[j][testBegin:testEnd]
        classTrain = classPaths[j][:testBegin]
        classTrain.extend(classPaths[j][testEnd:])
        classPathsTrain[j] = classTrain
        classPathsTest[j] = classTest
              
        paths.extend(classTrain)
        y.extend([groupClasses[j]]*len(classTrain))
        pathsTest.extend(classTest)
        yTest.extend([groupClasses[j]]*len(classTest))
    
    X = np.array([io.imread(file)[0,:,0:2].flatten()/255 for file in paths])
    XTest = np.array([io.imread(file)[0,:,0:2].flatten()/255 for file in pathsTest])
    
    #print(classPathsTest[6])
    #classScores[groupClasses[6]].append((np.array([io.imread(file)[0,:,0:2].flatten()/255 for file in classPathsTest[6]]),[groupClasses[6]]*len(classPathsTest[6])))
        
    
    classifier = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(X, y)
    testClassifier(classifier)
    nClassifier = 1
    classifier = GaussianNB().fit(X, y)
    testClassifier(classifier)
    nClassifier = 2
    classifier = DecisionTreeClassifier(random_state=0).fit(X, y)
    testClassifier(classifier)
    nClassifier = 3
    classifier = RandomForestClassifier(random_state=0).fit(X, y)
    testClassifier(classifier)
    nClassifier = 4
    classifier = make_pipeline(StandardScaler(), SVC(gamma='auto')).fit(X, y)
    testClassifier(classifier)
    nClassifier = 5
    classifier = KNeighborsClassifier().fit(X, y)
    testClassifier(classifier)
    for k in range(6):
        '''print(meanScores)
        print(scores)
        print(k)
        print(len(meanScores))
        print(len(scores))
        
        print(scores[k])'''
        meanScores[k]+=scores[k]
        for j in range(numGroups):
            meanClassScore[j][k] += classScores[j][k]
    '''print("________________________")
    print(meanScores)
    print("________________________")
    print("------------------------")
    print(meanClassScore)
    print("------------------------")
    print("++++++++++++++++++++++++")
    print(confusionMatrix)
    print("++++++++++++++++++++++++")
    print(wrongFiles)'''


for k in range(6):
    for j in range(numGroups):
        meanClassScore[j][k] /= folds
    meanScores[k] /= folds

print("\n\nResults")
for i in range(6):
    print(classifierNames[i]+":"+"\n\tAccuracy: {:.2f}".format(meanScores[i]))
    for j in range(numGroups):
        print("\tAccuracy for class"+groupNames[j]+": {:.2f}".format(meanClassScore[j][i]))
    print()
print("\n\nWrongly predicted files:")
for i in range(6):
    file= open(classifierNames[i]+"6_classes_wrongFiles.txt","w+")
    print(classifierNames[i]+":")
    for j in range(len(wrongFiles[i])):
        file.write(str(wrongFiles[i][j][0])+" correct:"+str(wrongFiles[i][j][1])+" predicted:"+str(wrongFiles[i][j][2])+"\n")
        print(wrongFiles[i][j])
    print()
    file.close()
    
print(confusionMatrix)
numClassifiers = len(classifierNames)
numRows = 2
fig, axes = plt.subplots(nrows=numRows, ncols=numClassifiers//numRows, figsize=(40,20),sharey=False)
for i in range(numClassifiers):
    ax = axes.flatten()[i]
    ax.set_title(classifierNames[i]+"\nAccuracy: {:.2f}".format(meanScores[i]), pad=20,fontdict={"weight":"bold"})
    arr = np.array([np.array(xi) for xi in confusionMatrix[i]])
    newArray = np.zeros(arr.shape, dtype=np.float32)
    sumArr = np.sum(arr, axis=1)
    for row in range(arr.shape[0]):
        newArray[row] = np.divide(arr[row],sumArr[row], casting='unsafe')
    ax.imshow(newArray, cmap ='Blues')
    ax.set_xticks(np.arange(numGroups))
    ax.set_yticks(np.arange(numGroups))
    ax.set_xticklabels(groupNames)
    ax.set_yticklabels(groupNames)
    for j in range(numGroups):
        for k in range(numGroups):
            
            #text = ax.text(k, j, "%.0f" % confusionMatrix[i][j][k],ha="center", va="center", color="black")
            text = ax.text(k, j, "%.0f" % arr[j][k],ha="center", va="center", color="black")

plt.tight_layout()
plt.show()


In [ ]:
meanClassScore = [[7.0, 7.0, 7.0, 7.0, 7.0, 7.0], [6.0, 4.1, 6.0, 6.0, 6.1, 4.1], [6.1, 3.3000000000000003, 6.1, 6.1, 5.1090909090909085, 5.199999999999999], [7.0, 5.090909090909091, 5.2, 6.0, 6.090909090909091, 6.090909090909091], [5.0, 5.444444444444445, 7.0, 7.0, 6.111111111111111, 7.0], [6.1, 5.1, 6.1, 6.1, 5.1, 5.290909090909091], [6.181818181818182, 1.481818181818182, 6.181818181818182, 7.0, 6.209090909090909, 6.181818181818182], [7.0, 4.945454545454545, 7.0, 7.0, 6.0, 7.0], [7.0, 1.0, 7.0, 7.0, 7.0, 6.0], [6.111111111111111, 4.777777777777777, 6.111111111111111, 6.111111111111111, 5.222222222222221, 5.444444444444445], [6.555555555555555, 3.944444444444444, 6.555555555555555, 6.555555555555555, 6.555555555555555, 6.555555555555555], [7.0, 1.4444444444444444, 7.0, 7.0, 5.1, 4.3], [6.4, 0.0, 5.672727272727273, 4.945454545454545, 4.945454545454545, 4.945454545454545], [6.5, 2.564102564102564, 6.083333333333333, 6.083333333333333, 5.314102564102564, 5.096153846153847], [6.384615384615384, 4.730769230769231, 4.230769230769231, 6.384615384615385, 5.923076923076923, 4.346153846153846], [4.6, 1.2181818181818183, 4.690909090909091, 5.327272727272727, 4.927272727272728, 3.509090909090909], [3.0, 2.888888888888889, 5.333333333333334, 5.333333333333334, 6.0, 6.0]]

confusionMatrix= [[[41, 32, 2, 0, 0, 0, 2, 0, 0, 0, 4, 0, 1, 2, 5, 1, 0], [21, 49, 0, 1, 0, 0, 0, 0, 0, 0, 7, 0, 1, 2, 8, 0, 0], [1, 1, 27, 22, 5, 9, 13, 0, 1, 0, 4, 0, 1, 2, 0, 1, 3], [1, 0, 8, 60, 3, 10, 4, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0], [2, 0, 18, 14, 7, 12, 15, 4, 6, 0, 0, 0, 3, 2, 1, 1, 5], [2, 0, 14, 37, 5, 17, 8, 0, 1, 0, 2, 0, 4, 0, 0, 0, 0], [0, 1, 5, 2, 0, 1, 61, 0, 2, 1, 1, 0, 1, 8, 1, 5, 1], [0, 2, 2, 1, 3, 0, 5, 6, 49, 6, 6, 0, 3, 1, 3, 0, 3], [1, 2, 5, 3, 4, 1, 9, 2, 53, 0, 1, 0, 2, 0, 1, 0, 0], [0, 1, 1, 5, 3, 5, 5, 4, 41, 7, 8, 0, 4, 1, 1, 1, 3], [1, 6, 1, 5, 0, 0, 0, 0, 1, 1, 62, 0, 2, 2, 8, 0, 1], [2, 5, 0, 4, 0, 3, 0, 3, 21, 2, 26, 2, 5, 4, 8, 1, 4], [0, 0, 1, 2, 0, 0, 2, 1, 1, 1, 11, 0, 14, 19, 21, 13, 4], [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 66, 5, 14, 2], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 2, 4, 74, 6, 0], [0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 18, 9, 56, 2], [0, 0, 5, 2, 0, 1, 17, 0, 0, 1, 8, 0, 5, 11, 2, 4, 34]], [[59, 20, 0, 3, 5, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0], [59, 11, 1, 2, 0, 0, 1, 1, 0, 0, 6, 0, 5, 2, 1, 0, 0], [4, 5, 6, 14, 16, 22, 5, 2, 2, 2, 5, 0, 0, 0, 1, 1, 5], [4, 3, 2, 36, 11, 23, 1, 1, 0, 1, 8, 0, 0, 0, 0, 0, 0], [2, 2, 2, 4, 25, 31, 5, 4, 2, 3, 2, 0, 3, 1, 0, 1, 3], [1, 1, 0, 18, 12, 45, 0, 1, 3, 3, 3, 0, 3, 0, 0, 0, 0], [0, 2, 0, 0, 13, 2, 36, 1, 2, 6, 0, 0, 3, 5, 1, 11, 8], [0, 1, 1, 2, 14, 4, 3, 6, 13, 26, 3, 1, 3, 0, 1, 0, 12], [0, 2, 3, 0, 8, 8, 3, 12, 26, 4, 0, 0, 5, 0, 1, 0, 12], [0, 0, 0, 2, 10, 7, 2, 6, 4, 35, 4, 0, 4, 1, 3, 0, 12], [7, 6, 0, 8, 4, 8, 0, 1, 0, 18, 34, 0, 2, 0, 0, 0, 2], [4, 3, 1, 8, 7, 7, 0, 1, 8, 22, 17, 0, 1, 2, 3, 0, 6], [0, 0, 1, 0, 6, 3, 2, 0, 0, 17, 7, 0, 15, 10, 14, 13, 2], [0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 3, 33, 22, 27, 0], [0, 2, 0, 0, 4, 2, 1, 0, 0, 1, 1, 0, 15, 15, 42, 6, 1], [0, 0, 0, 0, 3, 0, 5, 0, 0, 1, 0, 0, 3, 26, 7, 45, 0], [0, 1, 0, 3, 4, 7, 3, 2, 0, 11, 3, 0, 1, 4, 2, 5, 44]], [[52, 20, 2, 2, 2, 1, 0, 1, 2, 1, 6, 0, 0, 0, 0, 0, 1], [17, 53, 1, 0, 0, 0, 0, 0, 1, 0, 4, 5, 5, 0, 3, 0, 0], [2, 2, 24, 10, 16, 9, 6, 4, 4, 2, 4, 1, 2, 0, 2, 1, 1], [3, 0, 12, 34, 5, 17, 4, 2, 2, 0, 5, 3, 0, 0, 1, 0, 2], [1, 0, 10, 11, 23, 15, 10, 5, 2, 5, 1, 2, 3, 0, 1, 0, 1], [0, 1, 9, 18, 14, 28, 2, 3, 1, 5, 2, 3, 3, 0, 0, 0, 1], [1, 3, 4, 1, 5, 4, 35, 5, 5, 5, 0, 3, 0, 1, 3, 11, 4], [3, 0, 1, 1, 4, 2, 3, 25, 17, 14, 4, 9, 2, 1, 2, 0, 2], [1, 1, 1, 5, 7, 2, 5, 21, 17, 11, 0, 10, 0, 0, 0, 0, 3], [0, 1, 3, 3, 6, 4, 4, 10, 11, 24, 2, 14, 4, 1, 0, 1, 2], [3, 5, 3, 2, 1, 6, 1, 1, 0, 8, 36, 16, 4, 1, 0, 1, 2], [0, 3, 2, 3, 1, 3, 0, 8, 7, 14, 20, 12, 7, 1, 3, 1, 5], [0, 3, 4, 0, 2, 1, 1, 4, 0, 2, 3, 2, 28, 6, 16, 12, 6], [0, 1, 1, 0, 1, 0, 3, 0, 1, 0, 1, 0, 9, 33, 15, 23, 3], [0, 2, 1, 0, 2, 0, 1, 1, 0, 0, 1, 3, 12, 16, 39, 11, 1], [1, 0, 1, 0, 0, 0, 8, 0, 0, 0, 0, 1, 16, 27, 3, 33, 0], [1, 0, 1, 1, 2, 3, 6, 1, 1, 3, 2, 5, 4, 3, 5, 3, 49]], [[60, 19, 1, 0, 1, 0, 1, 1, 0, 0, 7, 0, 0, 0, 0, 0, 0], [21, 54, 0, 0, 1, 0, 0, 0, 0, 0, 4, 1, 1, 2, 5, 0, 0], [2, 0, 36, 9, 13, 12, 7, 0, 0, 0, 4, 1, 4, 0, 1, 0, 1], [2, 0, 3, 61, 2, 13, 5, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0], [2, 0, 12, 3, 38, 17, 7, 1, 1, 0, 0, 1, 2, 1, 0, 2, 3], [1, 0, 8, 27, 16, 27, 1, 0, 1, 2, 4, 0, 2, 0, 0, 0, 1], [0, 0, 5, 1, 10, 0, 50, 1, 2, 3, 0, 0, 0, 3, 2, 11, 2], [0, 0, 0, 1, 6, 0, 4, 32, 15, 19, 2, 8, 2, 0, 0, 0, 1], [1, 0, 3, 4, 5, 2, 5, 15, 27, 15, 1, 2, 1, 0, 0, 0, 3], [0, 1, 0, 1, 5, 0, 3, 11, 10, 40, 3, 11, 2, 0, 0, 0, 3], [2, 1, 0, 4, 4, 1, 0, 2, 0, 3, 53, 14, 4, 0, 0, 0, 2], [0, 5, 0, 0, 2, 1, 0, 10, 5, 17, 23, 17, 4, 1, 2, 1, 2], [0, 2, 1, 0, 0, 1, 1, 1, 0, 1, 4, 2, 42, 11, 10, 10, 4], [0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 6, 53, 13, 14, 1], [0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 6, 13, 54, 10, 1], [1, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 6, 27, 4, 47, 1], [0, 0, 2, 0, 0, 1, 3, 1, 0, 0, 1, 2, 5, 6, 2, 5, 62]], [[63, 14, 2, 1, 3, 0, 0, 0, 0, 0, 4, 0, 1, 0, 2, 0, 0], [27, 48, 0, 0, 1, 0, 1, 0, 1, 1, 4, 3, 1, 1, 1, 0, 0], [10, 0, 19, 12, 17, 12, 7, 1, 2, 1, 4, 1, 1, 0, 1, 0, 2], [6, 0, 10, 45, 4, 16, 4, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 13, 5, 31, 24, 9, 2, 0, 0, 0, 1, 1, 2, 0, 0, 2], [1, 0, 5, 18, 14, 36, 1, 0, 3, 3, 4, 2, 3, 0, 0, 0, 0], [1, 0, 2, 0, 15, 0, 54, 0, 2, 3, 0, 0, 1, 1, 0, 11, 0], [0, 0, 1, 0, 5, 0, 4, 31, 15, 25, 5, 4, 0, 0, 0, 0, 0], [1, 0, 2, 1, 7, 2, 1, 15, 33, 17, 1, 2, 1, 0, 0, 0, 1], [0, 0, 0, 1, 4, 0, 0, 6, 11, 50, 7, 6, 2, 0, 0, 0, 3], [3, 1, 3, 4, 1, 0, 0, 2, 0, 7, 58, 6, 2, 0, 0, 0, 3], [1, 3, 0, 1, 2, 0, 2, 6, 7, 24, 23, 13, 5, 0, 1, 0, 2], [1, 0, 2, 0, 1, 0, 1, 1, 0, 1, 7, 0, 38, 14, 11, 10, 3], [1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 7, 52, 12, 16, 1], [2, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 2, 4, 16, 51, 9, 2], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 5, 28, 6, 50, 0], [0, 0, 1, 0, 3, 0, 5, 4, 0, 2, 2, 1, 5, 4, 0, 5, 58]], [[43, 27, 3, 5, 3, 0, 0, 1, 0, 0, 4, 2, 2, 0, 0, 0, 0], [35, 33, 1, 4, 2, 0, 0, 0, 3, 2, 4, 2, 3, 0, 0, 0, 0], [5, 4, 16, 16, 12, 16, 4, 2, 2, 3, 3, 2, 2, 0, 0, 0, 3], [8, 5, 12, 25, 5, 12, 2, 3, 2, 2, 6, 0, 5, 0, 0, 0, 3], [3, 2, 21, 17, 8, 11, 2, 2, 4, 6, 1, 2, 5, 1, 2, 3, 0], [1, 3, 13, 19, 18, 9, 1, 3, 1, 6, 5, 5, 5, 0, 1, 0, 0], [2, 3, 3, 4, 10, 3, 22, 8, 4, 6, 0, 1, 3, 8, 2, 7, 4], [2, 7, 4, 6, 8, 4, 9, 14, 10, 14, 0, 2, 5, 0, 1, 0, 4], [3, 4, 8, 3, 3, 1, 9, 11, 10, 20, 1, 6, 2, 0, 0, 0, 3], [1, 2, 4, 5, 5, 4, 6, 12, 9, 20, 4, 6, 5, 0, 1, 0, 6], [6, 12, 9, 7, 2, 4, 0, 2, 1, 7, 23, 8, 6, 0, 1, 0, 2], [8, 4, 4, 7, 3, 6, 3, 5, 10, 14, 12, 4, 5, 2, 1, 0, 2], [3, 6, 5, 3, 5, 1, 2, 3, 0, 6, 2, 3, 13, 12, 12, 10, 4], [0, 2, 1, 1, 1, 2, 3, 0, 3, 1, 0, 1, 13, 28, 14, 20, 1], [1, 1, 1, 1, 3, 3, 0, 2, 0, 0, 2, 3, 11, 23, 31, 7, 1], [2, 0, 0, 2, 0, 0, 5, 3, 0, 1, 0, 0, 15, 25, 5, 31, 1], [0, 0, 8, 7, 8, 3, 7, 5, 0, 7, 3, 4, 6, 2, 1, 4, 25]]]

'''for k in range(6):
    for j in range(numClasses):
        meanClassScore[j][k] /= 7
    meanScores[k] /= 7

print("\n\nResults")
for i in range(6):
    print(classifierNames[i]+":"+"\n\tAccuracy: {:.2f}".format(meanScores[i]))
    for j in range(numClasses):
        print("\tAccuracy for class"+classes[j]+": {:.2f}".format(meanClassScore[j][i]))
    print()'''
    
    
numClassifiers = len(classifierNames)
numRows = 2
fig, axes = plt.subplots(nrows=numRows, ncols=numClassifiers//numRows, figsize=(40,20),sharey=False)
for i in range(numClassifiers):
    ax = axes.flatten()[i]
    ax.set_title(classifierNames[i]+"\nAccuracy: {:.2f}".format(meanScores[i]), pad=20,fontdict={"weight":"bold"})
    arr = np.array([np.array(xi) for xi in confusionMatrix[i]])
    newArray = np.zeros(arr.shape, dtype=np.float32)
    sumArr = np.sum(arr, axis=1)
    for row in range(arr.shape[0]):
        newArray[row] = np.divide(arr[row],sumArr[row], casting='unsafe')
    ax.imshow(newArray, cmap ='Blues')
    ax.set_xticks(np.arange(numClasses))
    ax.set_yticks(np.arange(numClasses))
    ax.set_xticklabels(classes)
    ax.set_yticklabels(classes)
    for j in range(numClasses):
        for k in range(numClasses):
            
            #text = ax.text(k, j, "%.0f" % confusionMatrix[i][j][k],ha="center", va="center", color="black")
            text = ax.text(k, j, "%.0f" % arr[j][k],ha="center", va="center", color="black")

plt.tight_layout()
plt.show()

In [ ]:
file = classPathsTest[3][0]
print(file)
classifier.predict((io.imread(file)[0,:,0:2].flatten()/255).reshape(1, -1))[0]

In [ ]:
def showFile(filePath):
    fileName = filePath.split("\\")[-1]
    parts = fileName.split("_")
    group = parts[0]
    bag = parts[1]
    piece = parts[2]
    side = parts[3].split(".")[0]
    path = './Ceramica/'+group+'/'+group+"_"+bag+"/1_"+side+'/capture/'
    
    maskPath = './pieces_data/masks/'+group+"_"+bag+"_"+side+'_labeledMask.png'
    paperMaskPath = './pieces_data/masks/'+group+"_"+bag+"_"+side+'_paperMask.png'
    imagePath = glob.glob(path+'*.hdr')[0].replace('\\','/')
    print(path,imagePath)
    baseImage = envi.open(imagePath)[:,:,:]
    Darkref = envi.open("./Ceramica/BW_Reference/capture/DARKREF_Im_ceramica_2020-12-15_14-54-13.hdr",
                        "./Ceramica/BW_Reference/capture/DARKREF_Im_ceramica_2020-12-15_14-54-13.raw")
    Whiteref = envi.open("./Ceramica/BW_Reference/capture/WHITEREF_Im_ceramica_2020-12-15_14-54-13.hdr",
                         "./Ceramica/BW_Reference/capture/WHITEREF_Im_ceramica_2020-12-15_14-54-13.raw")

    Darkref = np.repeat(Darkref[:,:,:],10, axis=0)
    Whiteref = np.repeat(Whiteref[:,:,:],10, axis=0)
    image = ((baseImage - Darkref)/(Whiteref - Darkref ))
    
    paperMask = io.imread(paperMaskPath)[:,:,0]
    PaperDark = Darkref[np.nonzero(paperMask)]
    PaperWhite =Whiteref[np.nonzero(paperMask)]
    Paper = ((baseImage[np.nonzero(paperMask)] - PaperDark)/(PaperWhite - PaperDark ))
    Paper = Paper.reshape(-1, Paper.shape[-1])
    paperMean = np.array([np.mean(Paper[:,x]) for x in range(Paper.shape[1])])
    image[:,:] /= paperMean
    
    
    maskImage = io.imread(maskPath)
    mask = maskImage[:,:,0]==int(piece)
    pieceImage = np.zeros((1000,512,224))
    pieceImage[np.nonzero(mask)]=image[np.nonzero(mask)]
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    pieceImage = pieceImage[rmin:rmax,cmin:cmax]
    #spectralImage = io.imread(filePath)
    #mean = spectralImage[0,:,0]/255
    #std = spectralImage[0,:,1]/255
    xValues = [397.66, 400.28, 402.9, 405.52, 408.13, 410.75, 413.37, 416.0, 418.62, 421.24, 423.86, 426.49, 429.12, 431.74, 434.37, 437.0, 439.63, 442.26, 444.89, 447.52, 450.16, 452.79, 455.43, 458.06, 460.7, 463.34, 465.98, 468.62, 471.26, 473.9, 476.54, 479.18, 481.83, 484.47, 487.12, 489.77, 492.42, 495.07, 497.72, 500.37, 503.02, 505.67, 508.32, 510.98, 513.63, 516.29, 518.95, 521.61, 524.27, 526.93, 529.59, 532.25, 534.91, 537.57, 540.24, 542.91, 545.57, 548.24, 550.91, 553.58, 556.25, 558.92, 561.59, 564.26, 566.94, 569.61, 572.29, 574.96, 577.64, 580.32, 583.0, 585.68, 588.36, 591.04, 593.73, 596.41, 599.1, 601.78, 604.47, 607.16, 609.85, 612.53, 615.23, 617.92, 620.61, 623.3, 626.0, 628.69, 631.39, 634.08, 636.78, 639.48, 642.18, 644.88, 647.58, 650.29, 652.99, 655.69, 658.4, 661.1, 663.81, 666.52, 669.23, 671.94, 674.65, 677.36, 680.07, 682.79, 685.5, 688.22, 690.93, 693.65, 696.37, 699.09, 701.81, 704.53, 707.25, 709.97, 712.7, 715.42, 718.15, 720.87, 723.6, 726.33, 729.06, 731.79, 734.52, 737.25, 739.98, 742.72, 745.45, 748.19, 750.93, 753.66, 756.4, 759.14, 761.88, 764.62, 767.36, 770.11, 772.85, 775.6, 778.34, 781.09, 783.84, 786.58, 789.33, 792.08, 794.84, 797.59, 800.34, 803.1, 805.85, 808.61, 811.36, 814.12, 816.88, 819.64, 822.4, 825.16, 827.92, 830.69, 833.45, 836.22, 838.98, 841.75, 844.52, 847.29, 850.06, 852.83, 855.6, 858.37, 861.14, 863.92, 866.69, 869.47, 872.25, 875.03, 877.8, 880.58, 883.37, 886.15, 888.93, 891.71, 894.5, 897.28, 900.07, 902.86, 905.64, 908.43, 911.22, 914.02, 916.81, 919.6, 922.39, 925.19, 927.98, 930.78, 933.58, 936.38, 939.18, 941.98, 944.78, 947.58, 950.38, 953.19, 955.99, 958.8, 961.6, 964.41, 967.22, 970.03, 972.84, 975.65, 978.46, 981.27, 984.09, 986.9, 989.72, 992.54, 995.35, 998.17, 1000.99, 1003.81]
    
    colors =   [[0,1,0], #tardoantiguitat
                [1,0,0], #roman
                [0,0,1], #iber
                [1,1,0]] #modern
    values = [[],[],[],[]]
    predictedImage = np.zeros((pieceImage.shape[0],pieceImage.shape[1],3))
    for y in range(1,predictedImage.shape[0]-1):
        for x in range(1,predictedImage.shape[1]-1):
            if mask[y+rmin,x+cmin] > 0.5:
                area = pieceImage[y-1:y+2,x-1:x+2]
                area = area.reshape(-1, area.shape[-1])
                std = np.std(area,axis=0)
                mean = pieceImage[y][x]
                pieceData = np.zeros((1,224,2))
                pieceData[0,:,0] = np.clip(mean,0,1)
                pieceData[0,:,1] = np.clip(std,0,1)
                #print(pieceData.shape)
                pieceData = pieceData.flatten()
                #print(pieceData.shape)
                pieceData = pieceData.reshape(1, -1)
                #print(pieceData.shape)
                #print(classifier.predict(pieceData))
                result = classifier.predict(pieceData)[0]
                values[result].append(pieceImage[y][x])
                predictedImage[y][x] = colors[result]
                #print("column " +str(x+1) +" of "+str(predictedImage.shape[1])+ " at row " +str(y))
        #print("row " +str(y+1) +" of "+str(predictedImage.shape[0]))
    value1 = np.array(values[1])
    value2 = np.array(values[2])
    value1mean = np.mean(value1,axis=0)
    value2mean = np.mean(value2,axis=0)
    value1std = np.std(value1,axis=0)
    value2std = np.std(value2,axis=0)
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharey=False, figsize=(9,3))
    ax1.imshow(image[:,:,[100,58,16]],vmin=0, vmax=1)
    ax2.imshow(pieceImage[:,:,[100,58,16]],vmin=0, vmax=1)
    ax3.imshow(predictedImage)
    ax4.plot(xValues,value1mean, color='r')
    ax4.fill_between(xValues,value1mean-value1std*2,value1mean+value1std*2, alpha = 0.2, color='r', label="Romana")
    ax4.plot(xValues,value2mean, color='b')
    ax4.fill_between(xValues,value2mean-value2std*2,value2mean+value2std*2, alpha = 0.2, color='b', label="Iberica")
    ax4.set_ylim([0,1])
    ax4.legend()

    #print(xValues[100],xValues[58],xValues[16])
    

In [ ]:
wrongFiles = [['./pieces_dataV3/spectrum\\gB_01_817_1A.png', './pieces_dataV3/spectrum\\gB_01_818_1A.png', './pieces_dataV3/spectrum\\gB_01_819_1A.png', './pieces_dataV3/spectrum\\gB_01_820_1A.png', './pieces_dataV3/spectrum\\gB_01_821_1A.png', './pieces_dataV3/spectrum\\gB_01_822_1A.png', './pieces_dataV3/spectrum\\gB_01_823_1A.png', './pieces_dataV3/spectrum\\gB_01_824_1A.png', './pieces_dataV3/spectrum\\gB_01_825_1A.png', './pieces_dataV3/spectrum\\gF_01_1211_1A.png', './pieces_dataV3/spectrum\\gF_01_1212_1A.png', './pieces_dataV3/spectrum\\gF_01_1213_1A.png', './pieces_dataV3/spectrum\\gF_01_1214_1A.png', './pieces_dataV3/spectrum\\gF_01_1215_1A.png', './pieces_dataV3/spectrum\\gF_01_1216_1A.png', './pieces_dataV3/spectrum\\gF_01_1217_1A.png', './pieces_dataV3/spectrum\\gF_01_1218_1A.png', './pieces_dataV3/spectrum\\gF_01_1219_1A.png', './pieces_dataV3/spectrum\\gK_01_1620_1A.png', './pieces_dataV3/spectrum\\gK_01_1621_1A.png', './pieces_dataV3/spectrum\\gK_01_1622_1A.png', './pieces_dataV3/spectrum\\gK_01_1623_1A.png', './pieces_dataV3/spectrum\\gK_01_1624_1A.png', './pieces_dataV3/spectrum\\gK_01_1625_1A.png', './pieces_dataV3/spectrum\\gK_01_1626_1A.png', './pieces_dataV3/spectrum\\gK_01_1627_1A.png', './pieces_dataV3/spectrum\\gN_01_1894_1A.png', './pieces_dataV3/spectrum\\gN_01_1895_1A.png', './pieces_dataV3/spectrum\\gN_01_1896_1A.png', './pieces_dataV3/spectrum\\gN_01_1897_1A.png', './pieces_dataV3/spectrum\\gN_01_1898_1A.png', './pieces_dataV3/spectrum\\gN_01_1899_1A.png', './pieces_dataV3/spectrum\\gP_01_2008_1B.png', './pieces_dataV3/spectrum\\gP_01_2009_1B.png', './pieces_dataV3/spectrum\\gP_01_2010_1B.png', './pieces_dataV3/spectrum\\gP_01_2011_1B.png', './pieces_dataV3/spectrum\\gP_01_2012_1B.png', './pieces_dataV3/spectrum\\gP_01_2013_1B.png', './pieces_dataV3/spectrum\\gR_01_2259_1A.png', './pieces_dataV3/spectrum\\gR_01_2260_1A.png', './pieces_dataV3/spectrum\\gR_01_2261_1A.png', './pieces_dataV3/spectrum\\gR_01_2262_1A.png', './pieces_dataV3/spectrum\\gS_01_2368_1A.png', './pieces_dataV3/spectrum\\gS_01_2369_1A.png', './pieces_dataV3/spectrum\\gS_01_2370_1A.png', './pieces_dataV3/spectrum\\gS_01_2371_1A.png', './pieces_dataV3/spectrum\\gS_01_2372_1A.png', './pieces_dataV3/spectrum\\gS_01_2373_1A.png', './pieces_dataV3/spectrum\\gS_01_2374_1A.png', './pieces_dataV3/spectrum\\gS_01_2375_1A.png', './pieces_dataV3/spectrum\\gR_01_2273_2A.png', './pieces_dataV3/spectrum\\gR_01_2274_2A.png', './pieces_dataV3/spectrum\\gR_01_2275_2A.png', './pieces_dataV3/spectrum\\gR_01_2276_2B.png', './pieces_dataV3/spectrum\\gR_01_2277_2B.png', './pieces_dataV3/spectrum\\gR_01_2278_2B.png', './pieces_dataV3/spectrum\\gQ_01_2162_2A.png', './pieces_dataV3/spectrum\\gQ_01_2163_2A.png', './pieces_dataV3/spectrum\\gQ_01_2164_2A.png', './pieces_dataV3/spectrum\\gQ_01_2165_2A.png', './pieces_dataV3/spectrum\\gQ_01_2166_2A.png', './pieces_dataV3/spectrum\\gC_01_929_1C.png', './pieces_dataV3/spectrum\\gC_01_930_1C.png', './pieces_dataV3/spectrum\\gC_01_931_1C.png', './pieces_dataV3/spectrum\\gC_01_932_1C.png', './pieces_dataV3/spectrum\\gC_01_933_1C.png', './pieces_dataV3/spectrum\\gC_01_934_1C.png', './pieces_dataV3/spectrum\\gC_01_935_1C.png', './pieces_dataV3/spectrum\\gC_01_936_1C.png', './pieces_dataV3/spectrum\\gC_01_937_1C.png', './pieces_dataV3/spectrum\\gR_01_2292_3B.png', './pieces_dataV3/spectrum\\gR_01_2293_3B.png', './pieces_dataV3/spectrum\\gR_01_2294_3B.png', './pieces_dataV3/spectrum\\gR_01_2295_3B.png', './pieces_dataV3/spectrum\\gR_01_2296_3B.png', './pieces_dataV3/spectrum\\gE_01_1155_2A.png', './pieces_dataV3/spectrum\\gE_01_1156_2A.png', './pieces_dataV3/spectrum\\gE_01_1157_2A.png', './pieces_dataV3/spectrum\\gE_01_1158_2A.png', './pieces_dataV3/spectrum\\gE_01_1159_2A.png', './pieces_dataV3/spectrum\\gE_01_1160_2A.png', './pieces_dataV3/spectrum\\gE_01_1161_2A.png', './pieces_dataV3/spectrum\\gE_01_1162_2B.png', './pieces_dataV3/spectrum\\gE_01_1163_2B.png', './pieces_dataV3/spectrum\\gE_01_1164_2B.png', './pieces_dataV3/spectrum\\gL_01_1745_2A.png', './pieces_dataV3/spectrum\\gL_01_1746_2A.png', './pieces_dataV3/spectrum\\gL_01_1747_2A.png', './pieces_dataV3/spectrum\\gL_01_1748_2A.png', './pieces_dataV3/spectrum\\gR_01_2304_4A.png', './pieces_dataV3/spectrum\\gR_01_2305_4A.png', './pieces_dataV3/spectrum\\gR_01_2306_4A.png', './pieces_dataV3/spectrum\\gR_01_2307_4A.png', './pieces_dataV3/spectrum\\gR_01_2308_4A.png', './pieces_dataV3/spectrum\\gS_01_2403_2B.png', './pieces_dataV3/spectrum\\gS_01_2404_2B.png', './pieces_dataV3/spectrum\\gS_01_2405_2B.png', './pieces_dataV3/spectrum\\gS_01_2406_2B.png', './pieces_dataV3/spectrum\\gS_01_2407_2B.png', './pieces_dataV3/spectrum\\gS_01_2408_2B.png', './pieces_dataV3/spectrum\\gS_01_2409_2B.png', './pieces_dataV3/spectrum\\gS_01_2410_2B.png', './pieces_dataV3/spectrum\\gS_01_2411_2B.png', './pieces_dataV3/spectrum\\gE_01_1165_2B.png', './pieces_dataV3/spectrum\\gE_01_1166_2B.png', './pieces_dataV3/spectrum\\gE_01_1167_2B.png', './pieces_dataV3/spectrum\\gE_01_1168_2B.png', './pieces_dataV3/spectrum\\gE_01_1169_2B.png', './pieces_dataV3/spectrum\\gE_01_1170_2B.png', './pieces_dataV3/spectrum\\gE_01_1171_2C.png', './pieces_dataV3/spectrum\\gE_01_1172_2C.png', './pieces_dataV3/spectrum\\gE_01_1173_2C.png', './pieces_dataV3/spectrum\\gR_01_2313_4C.png', './pieces_dataV3/spectrum\\gR_01_2314_4C.png', './pieces_dataV3/spectrum\\gR_01_2315_4C.png', './pieces_dataV3/spectrum\\gR_01_2316_4C.png', './pieces_dataV3/spectrum\\gR_01_2317_4C.png', './pieces_dataV3/spectrum\\gR_01_2318_4C.png', './pieces_dataV3/spectrum\\gS_01_2412_2B.png', './pieces_dataV3/spectrum\\gS_01_2413_2B.png', './pieces_dataV3/spectrum\\gS_01_2414_2B.png', './pieces_dataV3/spectrum\\gS_01_2415_2B.png', './pieces_dataV3/spectrum\\gS_01_2416_2B.png', './pieces_dataV3/spectrum\\gS_01_2417_2B.png', './pieces_dataV3/spectrum\\gS_01_2418_2B.png', './pieces_dataV3/spectrum\\gS_01_2419_2B.png', './pieces_dataV3/spectrum\\gG_01_1376_2C.png', './pieces_dataV3/spectrum\\gG_01_1377_2C.png', './pieces_dataV3/spectrum\\gG_01_1378_2C.png', './pieces_dataV3/spectrum\\gG_01_1379_2C.png', './pieces_dataV3/spectrum\\gG_01_1380_2C.png', './pieces_dataV3/spectrum\\gG_01_1381_2C.png', './pieces_dataV3/spectrum\\gG_01_1382_2C.png', './pieces_dataV3/spectrum\\gG_01_1383_2C.png', './pieces_dataV3/spectrum\\gG_01_1384_2C.png', './pieces_dataV3/spectrum\\gQ_01_2211_4B.png', './pieces_dataV3/spectrum\\gQ_01_2212_4B.png', './pieces_dataV3/spectrum\\gQ_01_2213_4B.png', './pieces_dataV3/spectrum\\gS_01_2420_2B.png', './pieces_dataV3/spectrum\\gS_01_2421_2B.png', './pieces_dataV3/spectrum\\gS_01_2422_2B.png', './pieces_dataV3/spectrum\\gS_01_2423_2B.png', './pieces_dataV3/spectrum\\gS_01_2424_2B.png', './pieces_dataV3/spectrum\\gS_01_2425_2B.png', './pieces_dataV3/spectrum\\gS_01_2426_2C.png', './pieces_dataV3/spectrum\\gS_01_2427_2C.png', './pieces_dataV3/spectrum\\gS_01_2428_2C.png', './pieces_dataV3/spectrum\\gP_01_2098_4C.png', './pieces_dataV3/spectrum\\gP_01_2099_4C.png', './pieces_dataV3/spectrum\\gP_01_2100_4C.png', './pieces_dataV3/spectrum\\gP_01_2101_4C.png', './pieces_dataV3/spectrum\\gP_01_2102_4C.png', './pieces_dataV3/spectrum\\gP_01_2103_4C.png', './pieces_dataV3/spectrum\\gQ_01_2223_5B.png', './pieces_dataV3/spectrum\\gQ_01_2224_5B.png', './pieces_dataV3/spectrum\\gQ_01_2225_5B.png', './pieces_dataV3/spectrum\\gS_01_2429_2C.png', './pieces_dataV3/spectrum\\gS_01_2430_2C.png', './pieces_dataV3/spectrum\\gS_01_2431_2C.png', './pieces_dataV3/spectrum\\gS_01_2432_2C.png', './pieces_dataV3/spectrum\\gS_01_2433_2C.png', './pieces_dataV3/spectrum\\gS_01_2434_3A.png', './pieces_dataV3/spectrum\\gS_01_2435_3A.png', './pieces_dataV3/spectrum\\gS_01_2436_3A.png', './pieces_dataV3/spectrum\\gS_01_2437_3A.png', './pieces_dataV3/spectrum\\gE_01_1192_3B.png', './pieces_dataV3/spectrum\\gE_01_1193_3B.png', './pieces_dataV3/spectrum\\gE_01_1194_3B.png', './pieces_dataV3/spectrum\\gE_01_1195_3B.png', './pieces_dataV3/spectrum\\gE_01_1196_3B.png', './pieces_dataV3/spectrum\\gE_01_1197_3B.png', './pieces_dataV3/spectrum\\gE_01_1198_3B.png', './pieces_dataV3/spectrum\\gE_01_1199_3B.png', './pieces_dataV3/spectrum\\gE_01_1200_3B.png', './pieces_dataV3/spectrum\\gL_01_1779_3B.png', './pieces_dataV3/spectrum\\gL_01_1780_3B.png', './pieces_dataV3/spectrum\\gL_01_1781_3B.png', './pieces_dataV3/spectrum\\gL_01_1782_3B.png', './pieces_dataV3/spectrum\\gL_01_1783_3B.png', './pieces_dataV3/spectrum\\gL_01_1784_3B.png', './pieces_dataV3/spectrum\\gL_01_1785_3B.png', './pieces_dataV3/spectrum\\gL_01_1786_3B.png', './pieces_dataV3/spectrum\\gM_01_1874_3B.png', './pieces_dataV3/spectrum\\gM_01_1875_3B.png', './pieces_dataV3/spectrum\\gM_01_1876_3B.png', './pieces_dataV3/spectrum\\gM_01_1877_3B.png', './pieces_dataV3/spectrum\\gM_01_1878_3B.png', './pieces_dataV3/spectrum\\gM_01_1879_3B.png', './pieces_dataV3/spectrum\\gM_01_1880_3B.png', './pieces_dataV3/spectrum\\gR_01_2349_7A.png', './pieces_dataV3/spectrum\\gR_01_2350_7A.png', './pieces_dataV3/spectrum\\gR_01_2351_7A.png', './pieces_dataV3/spectrum\\gR_01_2352_7A.png', './pieces_dataV3/spectrum\\gS_01_2438_3A.png', './pieces_dataV3/spectrum\\gS_01_2439_3B.png', './pieces_dataV3/spectrum\\gS_01_2440_3B.png', './pieces_dataV3/spectrum\\gS_01_2441_3B.png', './pieces_dataV3/spectrum\\gS_01_2442_3B.png', './pieces_dataV3/spectrum\\gS_01_2443_3B.png', './pieces_dataV3/spectrum\\gS_01_2444_3B.png', './pieces_dataV3/spectrum\\gS_01_2445_3C.png', './pieces_dataV3/spectrum\\gS_01_2446_3C.png', './pieces_dataV3/spectrum\\gE_01_1202_3C.png', './pieces_dataV3/spectrum\\gE_01_1203_3C.png', './pieces_dataV3/spectrum\\gE_01_1204_3C.png', './pieces_dataV3/spectrum\\gE_01_1205_3C.png', './pieces_dataV3/spectrum\\gE_01_1206_3C.png', './pieces_dataV3/spectrum\\gE_01_1207_3C.png', './pieces_dataV3/spectrum\\gE_01_1208_3C.png', './pieces_dataV3/spectrum\\gE_01_1209_3C.png', './pieces_dataV3/spectrum\\gE_01_1210_3C.png', './pieces_dataV3/spectrum\\gF_01_1304_3C.png', './pieces_dataV3/spectrum\\gF_01_1305_3C.png', './pieces_dataV3/spectrum\\gF_01_1306_3C.png', './pieces_dataV3/spectrum\\gF_01_1307_3C.png', './pieces_dataV3/spectrum\\gF_01_1308_3C.png', './pieces_dataV3/spectrum\\gF_01_1309_3C.png', './pieces_dataV3/spectrum\\gF_01_1310_3C.png', './pieces_dataV3/spectrum\\gF_01_1311_3C.png', './pieces_dataV3/spectrum\\gF_01_1312_3C.png', './pieces_dataV3/spectrum\\gF_01_1313_3C.png', './pieces_dataV3/spectrum\\gG_01_1407_3C.png', './pieces_dataV3/spectrum\\gG_01_1408_3C.png', './pieces_dataV3/spectrum\\gG_01_1409_3C.png', './pieces_dataV3/spectrum\\gG_01_1410_3C.png', './pieces_dataV3/spectrum\\gG_01_1411_3C.png', './pieces_dataV3/spectrum\\gG_01_1412_3C.png', './pieces_dataV3/spectrum\\gG_01_1413_3C.png', './pieces_dataV3/spectrum\\gG_01_1414_3C.png', './pieces_dataV3/spectrum\\gG_01_1415_3C.png', './pieces_dataV3/spectrum\\gG_01_1416_3C.png', './pieces_dataV3/spectrum\\gL_01_1787_3C.png', './pieces_dataV3/spectrum\\gL_01_1788_3C.png', './pieces_dataV3/spectrum\\gL_01_1789_3C.png', './pieces_dataV3/spectrum\\gL_01_1790_3C.png', './pieces_dataV3/spectrum\\gL_01_1791_3C.png', './pieces_dataV3/spectrum\\gL_01_1792_3C.png', './pieces_dataV3/spectrum\\gL_01_1793_3C.png', './pieces_dataV3/spectrum\\gL_01_1794_3C.png', './pieces_dataV3/spectrum\\gL_01_1795_3C.png', './pieces_dataV3/spectrum\\gN_01_1996_4C.png', './pieces_dataV3/spectrum\\gN_01_1997_4C.png', './pieces_dataV3/spectrum\\gN_01_1998_4C.png', './pieces_dataV3/spectrum\\gN_01_1999_4C.png', './pieces_dataV3/spectrum\\gN_01_2000_4C.png', './pieces_dataV3/spectrum\\gN_01_2001_4C.png', './pieces_dataV3/spectrum\\gN_01_2002_4C.png', './pieces_dataV3/spectrum\\gR_01_2366_8C.png', './pieces_dataV3/spectrum\\gR_01_2367_8C.png', './pieces_dataV3/spectrum\\gS_01_2447_3C.png', './pieces_dataV3/spectrum\\gS_01_2448_3C.png', './pieces_dataV3/spectrum\\gS_01_2449_3C.png', './pieces_dataV3/spectrum\\gS_01_2450_3C.png', './pieces_dataV3/spectrum\\gS_01_2451_3C.png', './pieces_dataV3/spectrum\\gS_01_2452_3C.png', './pieces_dataV3/spectrum\\gS_01_2453_3C.png', './pieces_dataV3/spectrum\\gS_01_2454_3C.png', './pieces_dataV3/spectrum\\gS_01_2455_3C.png'], ['./pieces_dataV3/spectrum\\gB_01_817_1A.png', './pieces_dataV3/spectrum\\gB_01_818_1A.png', './pieces_dataV3/spectrum\\gB_01_819_1A.png', './pieces_dataV3/spectrum\\gB_01_820_1A.png', './pieces_dataV3/spectrum\\gB_01_821_1A.png', './pieces_dataV3/spectrum\\gB_01_822_1A.png', './pieces_dataV3/spectrum\\gB_01_823_1A.png', './pieces_dataV3/spectrum\\gB_01_824_1A.png', './pieces_dataV3/spectrum\\gB_01_825_1A.png', './pieces_dataV3/spectrum\\gF_01_1211_1A.png', './pieces_dataV3/spectrum\\gF_01_1212_1A.png', './pieces_dataV3/spectrum\\gF_01_1213_1A.png', './pieces_dataV3/spectrum\\gF_01_1214_1A.png', './pieces_dataV3/spectrum\\gF_01_1215_1A.png', './pieces_dataV3/spectrum\\gF_01_1216_1A.png', './pieces_dataV3/spectrum\\gF_01_1217_1A.png', './pieces_dataV3/spectrum\\gF_01_1218_1A.png', './pieces_dataV3/spectrum\\gF_01_1219_1A.png', './pieces_dataV3/spectrum\\gJ_01_1519_1A.png', './pieces_dataV3/spectrum\\gJ_01_1520_1A.png', './pieces_dataV3/spectrum\\gJ_01_1521_1A.png', './pieces_dataV3/spectrum\\gJ_01_1522_1A.png', './pieces_dataV3/spectrum\\gJ_01_1523_1A.png', './pieces_dataV3/spectrum\\gJ_01_1524_1A.png', './pieces_dataV3/spectrum\\gJ_01_1525_1A.png', './pieces_dataV3/spectrum\\gJ_01_1526_1A.png', './pieces_dataV3/spectrum\\gJ_01_1527_1A.png', './pieces_dataV3/spectrum\\gJ_01_1528_1A.png', './pieces_dataV3/spectrum\\gK_01_1620_1A.png', './pieces_dataV3/spectrum\\gK_01_1621_1A.png', './pieces_dataV3/spectrum\\gK_01_1622_1A.png', './pieces_dataV3/spectrum\\gK_01_1623_1A.png', './pieces_dataV3/spectrum\\gK_01_1624_1A.png', './pieces_dataV3/spectrum\\gK_01_1625_1A.png', './pieces_dataV3/spectrum\\gK_01_1626_1A.png', './pieces_dataV3/spectrum\\gK_01_1627_1A.png', './pieces_dataV3/spectrum\\gM_01_1796_1A.png', './pieces_dataV3/spectrum\\gM_01_1797_1A.png', './pieces_dataV3/spectrum\\gM_01_1798_1A.png', './pieces_dataV3/spectrum\\gM_01_1799_1A.png', './pieces_dataV3/spectrum\\gM_01_1800_1A.png', './pieces_dataV3/spectrum\\gM_01_1801_1A.png', './pieces_dataV3/spectrum\\gM_01_1802_1A.png', './pieces_dataV3/spectrum\\gM_01_1803_1A.png', './pieces_dataV3/spectrum\\gM_01_1804_1A.png', './pieces_dataV3/spectrum\\gN_01_1894_1A.png', './pieces_dataV3/spectrum\\gN_01_1895_1A.png', './pieces_dataV3/spectrum\\gN_01_1896_1A.png', './pieces_dataV3/spectrum\\gN_01_1897_1A.png', './pieces_dataV3/spectrum\\gN_01_1898_1A.png', './pieces_dataV3/spectrum\\gN_01_1899_1A.png', './pieces_dataV3/spectrum\\gN_01_1900_1B.png', './pieces_dataV3/spectrum\\gN_01_1901_1B.png', './pieces_dataV3/spectrum\\gN_01_1902_1B.png', './pieces_dataV3/spectrum\\gN_01_1903_1B.png', './pieces_dataV3/spectrum\\gP_01_2003_1A.png', './pieces_dataV3/spectrum\\gP_01_2004_1A.png', './pieces_dataV3/spectrum\\gP_01_2005_1A.png', './pieces_dataV3/spectrum\\gP_01_2006_1A.png', './pieces_dataV3/spectrum\\gP_01_2007_1A.png', './pieces_dataV3/spectrum\\gP_01_2008_1B.png', './pieces_dataV3/spectrum\\gP_01_2009_1B.png', './pieces_dataV3/spectrum\\gP_01_2010_1B.png', './pieces_dataV3/spectrum\\gP_01_2011_1B.png', './pieces_dataV3/spectrum\\gP_01_2012_1B.png', './pieces_dataV3/spectrum\\gP_01_2013_1B.png', './pieces_dataV3/spectrum\\gR_01_2259_1A.png', './pieces_dataV3/spectrum\\gR_01_2260_1A.png', './pieces_dataV3/spectrum\\gR_01_2261_1A.png', './pieces_dataV3/spectrum\\gR_01_2262_1A.png', './pieces_dataV3/spectrum\\gR_01_2267_1C.png', './pieces_dataV3/spectrum\\gR_01_2268_1C.png', './pieces_dataV3/spectrum\\gS_01_2368_1A.png', './pieces_dataV3/spectrum\\gS_01_2369_1A.png', './pieces_dataV3/spectrum\\gS_01_2370_1A.png', './pieces_dataV3/spectrum\\gS_01_2371_1A.png', './pieces_dataV3/spectrum\\gS_01_2372_1A.png', './pieces_dataV3/spectrum\\gS_01_2373_1A.png', './pieces_dataV3/spectrum\\gS_01_2374_1A.png', './pieces_dataV3/spectrum\\gS_01_2375_1A.png', './pieces_dataV3/spectrum\\gB_01_826_1B.png', './pieces_dataV3/spectrum\\gB_01_827_1B.png', './pieces_dataV3/spectrum\\gB_01_828_1B.png', './pieces_dataV3/spectrum\\gB_01_829_1B.png', './pieces_dataV3/spectrum\\gB_01_830_1B.png', './pieces_dataV3/spectrum\\gB_01_831_1B.png', './pieces_dataV3/spectrum\\gB_01_832_1B.png', './pieces_dataV3/spectrum\\gB_01_833_1B.png', './pieces_dataV3/spectrum\\gB_01_834_1B.png', './pieces_dataV3/spectrum\\gD_01_1025_1B.png', './pieces_dataV3/spectrum\\gD_01_1026_1B.png', './pieces_dataV3/spectrum\\gD_01_1027_1B.png', './pieces_dataV3/spectrum\\gD_01_1028_1B.png', './pieces_dataV3/spectrum\\gD_01_1029_1B.png', './pieces_dataV3/spectrum\\gD_01_1030_1B.png', './pieces_dataV3/spectrum\\gD_01_1031_1B.png', './pieces_dataV3/spectrum\\gD_01_1032_1B.png', './pieces_dataV3/spectrum\\gD_01_1033_1C.png', './pieces_dataV3/spectrum\\gD_01_1034_1C.png', './pieces_dataV3/spectrum\\gE_01_1131_1B.png', './pieces_dataV3/spectrum\\gE_01_1132_1B.png', './pieces_dataV3/spectrum\\gE_01_1133_1B.png', './pieces_dataV3/spectrum\\gE_01_1134_1B.png', './pieces_dataV3/spectrum\\gE_01_1135_1B.png', './pieces_dataV3/spectrum\\gE_01_1136_1B.png', './pieces_dataV3/spectrum\\gF_01_1229_1C.png', './pieces_dataV3/spectrum\\gF_01_1230_1C.png', './pieces_dataV3/spectrum\\gG_01_1327_1B.png', './pieces_dataV3/spectrum\\gG_01_1328_1B.png', './pieces_dataV3/spectrum\\gG_01_1329_1B.png', './pieces_dataV3/spectrum\\gG_01_1330_1B.png', './pieces_dataV3/spectrum\\gG_01_1331_1B.png', './pieces_dataV3/spectrum\\gG_01_1332_1B.png', './pieces_dataV3/spectrum\\gG_01_1333_1B.png', './pieces_dataV3/spectrum\\gL_01_1721_1B.png', './pieces_dataV3/spectrum\\gL_01_1722_1B.png', './pieces_dataV3/spectrum\\gL_01_1723_1B.png', './pieces_dataV3/spectrum\\gL_01_1724_1B.png', './pieces_dataV3/spectrum\\gL_01_1725_1B.png', './pieces_dataV3/spectrum\\gL_01_1726_1B.png', './pieces_dataV3/spectrum\\gM_01_1805_1A.png', './pieces_dataV3/spectrum\\gM_01_1806_1A.png', './pieces_dataV3/spectrum\\gM_01_1807_1B.png', './pieces_dataV3/spectrum\\gM_01_1808_1B.png', './pieces_dataV3/spectrum\\gM_01_1809_1B.png', './pieces_dataV3/spectrum\\gM_01_1810_1B.png', './pieces_dataV3/spectrum\\gM_01_1811_1B.png', './pieces_dataV3/spectrum\\gM_01_1812_1B.png', './pieces_dataV3/spectrum\\gM_01_1813_1B.png', './pieces_dataV3/spectrum\\gM_01_1814_1B.png', './pieces_dataV3/spectrum\\gN_01_1904_1B.png', './pieces_dataV3/spectrum\\gN_01_1905_1B.png', './pieces_dataV3/spectrum\\gN_01_1906_1B.png', './pieces_dataV3/spectrum\\gN_01_1907_1B.png', './pieces_dataV3/spectrum\\gN_01_1908_1B.png', './pieces_dataV3/spectrum\\gN_01_1909_1B.png', './pieces_dataV3/spectrum\\gN_01_1910_1B.png', './pieces_dataV3/spectrum\\gN_01_1911_1B.png', './pieces_dataV3/spectrum\\gN_01_1912_1C.png', './pieces_dataV3/spectrum\\gN_01_1913_1C.png', './pieces_dataV3/spectrum\\gN_01_1914_1C.png', './pieces_dataV3/spectrum\\gP_01_2027_2B.png', './pieces_dataV3/spectrum\\gR_01_2269_1C.png', './pieces_dataV3/spectrum\\gR_01_2270_1C.png', './pieces_dataV3/spectrum\\gR_01_2271_1C.png', './pieces_dataV3/spectrum\\gR_01_2272_1C.png', './pieces_dataV3/spectrum\\gR_01_2273_2A.png', './pieces_dataV3/spectrum\\gR_01_2274_2A.png', './pieces_dataV3/spectrum\\gR_01_2275_2A.png', './pieces_dataV3/spectrum\\gR_01_2276_2B.png', './pieces_dataV3/spectrum\\gR_01_2277_2B.png', './pieces_dataV3/spectrum\\gR_01_2278_2B.png', './pieces_dataV3/spectrum\\gS_01_2376_1B.png', './pieces_dataV3/spectrum\\gS_01_2377_1B.png', './pieces_dataV3/spectrum\\gS_01_2378_1B.png', './pieces_dataV3/spectrum\\gS_01_2379_1B.png', './pieces_dataV3/spectrum\\gS_01_2380_1B.png', './pieces_dataV3/spectrum\\gS_01_2381_1B.png', './pieces_dataV3/spectrum\\gS_01_2382_1B.png', './pieces_dataV3/spectrum\\gS_01_2383_1B.png', './pieces_dataV3/spectrum\\gS_01_2384_1B.png', './pieces_dataV3/spectrum\\gC_01_919_1B.png', './pieces_dataV3/spectrum\\gC_01_920_1B.png', './pieces_dataV3/spectrum\\gC_01_921_1B.png', './pieces_dataV3/spectrum\\gC_01_922_1B.png', './pieces_dataV3/spectrum\\gC_01_923_1B.png', './pieces_dataV3/spectrum\\gC_01_924_1B.png', './pieces_dataV3/spectrum\\gC_01_925_1B.png', './pieces_dataV3/spectrum\\gC_01_926_1B.png', './pieces_dataV3/spectrum\\gC_01_927_1B.png', './pieces_dataV3/spectrum\\gD_01_1035_1C.png', './pieces_dataV3/spectrum\\gD_01_1036_1C.png', './pieces_dataV3/spectrum\\gD_01_1037_1C.png', './pieces_dataV3/spectrum\\gD_01_1038_1C.png', './pieces_dataV3/spectrum\\gD_01_1039_1C.png', './pieces_dataV3/spectrum\\gD_01_1040_1C.png', './pieces_dataV3/spectrum\\gD_01_1041_1C.png', './pieces_dataV3/spectrum\\gD_01_1042_1C.png', './pieces_dataV3/spectrum\\gD_01_1043_1C.png', './pieces_dataV3/spectrum\\gD_01_1044_1C.png', './pieces_dataV3/spectrum\\gE_01_1137_1B.png', './pieces_dataV3/spectrum\\gE_01_1138_1B.png', './pieces_dataV3/spectrum\\gE_01_1139_1B.png', './pieces_dataV3/spectrum\\gE_01_1140_1B.png', './pieces_dataV3/spectrum\\gE_01_1141_1B.png', './pieces_dataV3/spectrum\\gF_01_1231_1C.png', './pieces_dataV3/spectrum\\gF_01_1232_1C.png', './pieces_dataV3/spectrum\\gF_01_1233_1C.png', './pieces_dataV3/spectrum\\gF_01_1234_1C.png', './pieces_dataV3/spectrum\\gF_01_1235_1C.png', './pieces_dataV3/spectrum\\gF_01_1236_1C.png', './pieces_dataV3/spectrum\\gF_01_1237_1C.png', './pieces_dataV3/spectrum\\gF_01_1238_1C.png', './pieces_dataV3/spectrum\\gG_01_1334_1B.png', './pieces_dataV3/spectrum\\gG_01_1335_1B.png', './pieces_dataV3/spectrum\\gG_01_1336_1B.png', './pieces_dataV3/spectrum\\gG_01_1337_1B.png', './pieces_dataV3/spectrum\\gG_01_1338_1B.png', './pieces_dataV3/spectrum\\gG_01_1339_1B.png', './pieces_dataV3/spectrum\\gG_01_1340_1B.png', './pieces_dataV3/spectrum\\gG_01_1341_1B.png', './pieces_dataV3/spectrum\\gG_01_1342_1B.png', './pieces_dataV3/spectrum\\gG_01_1343_1B.png', './pieces_dataV3/spectrum\\gJ_01_1539_1B.png', './pieces_dataV3/spectrum\\gJ_01_1540_1B.png', './pieces_dataV3/spectrum\\gJ_01_1541_1B.png', './pieces_dataV3/spectrum\\gJ_01_1542_1B.png', './pieces_dataV3/spectrum\\gJ_01_1543_1B.png', './pieces_dataV3/spectrum\\gJ_01_1544_1B.png', './pieces_dataV3/spectrum\\gJ_01_1545_1B.png', './pieces_dataV3/spectrum\\gJ_01_1546_1B.png', './pieces_dataV3/spectrum\\gJ_01_1547_1B.png', './pieces_dataV3/spectrum\\gJ_01_1548_1B.png', './pieces_dataV3/spectrum\\gL_01_1727_1B.png', './pieces_dataV3/spectrum\\gL_01_1728_1B.png', './pieces_dataV3/spectrum\\gL_01_1729_1B.png', './pieces_dataV3/spectrum\\gL_01_1730_1B.png', './pieces_dataV3/spectrum\\gL_01_1731_1B.png', './pieces_dataV3/spectrum\\gM_01_1815_1B.png', './pieces_dataV3/spectrum\\gM_01_1816_1C.png', './pieces_dataV3/spectrum\\gM_01_1817_1C.png', './pieces_dataV3/spectrum\\gM_01_1818_1C.png', './pieces_dataV3/spectrum\\gM_01_1819_1C.png', './pieces_dataV3/spectrum\\gM_01_1820_1C.png', './pieces_dataV3/spectrum\\gM_01_1821_1C.png', './pieces_dataV3/spectrum\\gM_01_1822_1C.png', './pieces_dataV3/spectrum\\gM_01_1823_1C.png', './pieces_dataV3/spectrum\\gM_01_1824_1C.png', './pieces_dataV3/spectrum\\gN_01_1915_1C.png', './pieces_dataV3/spectrum\\gN_01_1916_1C.png', './pieces_dataV3/spectrum\\gN_01_1917_1C.png', './pieces_dataV3/spectrum\\gN_01_1918_1C.png', './pieces_dataV3/spectrum\\gN_01_1919_1C.png', './pieces_dataV3/spectrum\\gN_01_1920_1C.png', './pieces_dataV3/spectrum\\gN_01_1921_1C.png', './pieces_dataV3/spectrum\\gN_01_1922_2A.png', './pieces_dataV3/spectrum\\gN_01_1923_2A.png', './pieces_dataV3/spectrum\\gN_01_1924_2A.png', './pieces_dataV3/spectrum\\gN_01_1925_2A.png', './pieces_dataV3/spectrum\\gP_01_2028_2B.png', './pieces_dataV3/spectrum\\gP_01_2029_2C.png', './pieces_dataV3/spectrum\\gP_01_2030_2C.png', './pieces_dataV3/spectrum\\gP_01_2031_2C.png', './pieces_dataV3/spectrum\\gP_01_2032_2C.png', './pieces_dataV3/spectrum\\gP_01_2033_2C.png', './pieces_dataV3/spectrum\\gR_01_2280_2C.png', './pieces_dataV3/spectrum\\gR_01_2281_2C.png', './pieces_dataV3/spectrum\\gR_01_2282_2C.png', './pieces_dataV3/spectrum\\gR_01_2283_2C.png', './pieces_dataV3/spectrum\\gR_01_2284_2C.png', './pieces_dataV3/spectrum\\gR_01_2285_3A.png', './pieces_dataV3/spectrum\\gR_01_2286_3A.png', './pieces_dataV3/spectrum\\gR_01_2287_3A.png', './pieces_dataV3/spectrum\\gR_01_2288_3A.png', './pieces_dataV3/spectrum\\gR_01_2289_3A.png', './pieces_dataV3/spectrum\\gR_01_2290_3A.png', './pieces_dataV3/spectrum\\gS_01_2385_1B.png', './pieces_dataV3/spectrum\\gS_01_2386_1C.png', './pieces_dataV3/spectrum\\gS_01_2387_1C.png', './pieces_dataV3/spectrum\\gS_01_2388_1C.png', './pieces_dataV3/spectrum\\gS_01_2389_1C.png', './pieces_dataV3/spectrum\\gS_01_2390_1C.png', './pieces_dataV3/spectrum\\gS_01_2391_1C.png', './pieces_dataV3/spectrum\\gS_01_2392_1C.png', './pieces_dataV3/spectrum\\gS_01_2393_1C.png', './pieces_dataV3/spectrum\\gB_01_846_2A.png', './pieces_dataV3/spectrum\\gB_01_847_2A.png', './pieces_dataV3/spectrum\\gB_01_848_2A.png', './pieces_dataV3/spectrum\\gB_01_849_2A.png', './pieces_dataV3/spectrum\\gB_01_850_2A.png', './pieces_dataV3/spectrum\\gB_01_851_2A.png', './pieces_dataV3/spectrum\\gB_01_852_2A.png', './pieces_dataV3/spectrum\\gB_01_853_2A.png', './pieces_dataV3/spectrum\\gB_01_854_2A.png', './pieces_dataV3/spectrum\\gC_01_929_1C.png', './pieces_dataV3/spectrum\\gC_01_930_1C.png', './pieces_dataV3/spectrum\\gC_01_931_1C.png', './pieces_dataV3/spectrum\\gC_01_932_1C.png', './pieces_dataV3/spectrum\\gC_01_933_1C.png', './pieces_dataV3/spectrum\\gC_01_934_1C.png', './pieces_dataV3/spectrum\\gC_01_935_1C.png', './pieces_dataV3/spectrum\\gC_01_936_1C.png', './pieces_dataV3/spectrum\\gC_01_937_1C.png', './pieces_dataV3/spectrum\\gG_01_1346_1C.png', './pieces_dataV3/spectrum\\gG_01_1347_1C.png', './pieces_dataV3/spectrum\\gG_01_1348_1C.png', './pieces_dataV3/spectrum\\gG_01_1349_1C.png', './pieces_dataV3/spectrum\\gG_01_1350_1C.png', './pieces_dataV3/spectrum\\gG_01_1351_1C.png', './pieces_dataV3/spectrum\\gG_01_1352_1C.png', './pieces_dataV3/spectrum\\gG_01_1353_1C.png', './pieces_dataV3/spectrum\\gG_01_1354_1C.png', './pieces_dataV3/spectrum\\gJ_01_1549_1B.png', './pieces_dataV3/spectrum\\gJ_01_1550_1B.png', './pieces_dataV3/spectrum\\gJ_01_1551_1C.png', './pieces_dataV3/spectrum\\gJ_01_1552_1C.png', './pieces_dataV3/spectrum\\gJ_01_1553_1C.png', './pieces_dataV3/spectrum\\gJ_01_1554_1C.png', './pieces_dataV3/spectrum\\gJ_01_1555_1C.png', './pieces_dataV3/spectrum\\gJ_01_1556_1C.png', './pieces_dataV3/spectrum\\gJ_01_1557_1C.png', './pieces_dataV3/spectrum\\gJ_01_1558_1C.png', './pieces_dataV3/spectrum\\gM_01_1825_1C.png', './pieces_dataV3/spectrum\\gM_01_1826_1C.png', './pieces_dataV3/spectrum\\gM_01_1827_2A.png', './pieces_dataV3/spectrum\\gM_01_1828_2A.png', './pieces_dataV3/spectrum\\gM_01_1829_2A.png', './pieces_dataV3/spectrum\\gM_01_1830_2A.png', './pieces_dataV3/spectrum\\gM_01_1831_2A.png', './pieces_dataV3/spectrum\\gM_01_1832_2A.png', './pieces_dataV3/spectrum\\gM_01_1833_2A.png', './pieces_dataV3/spectrum\\gM_01_1834_2A.png', './pieces_dataV3/spectrum\\gN_01_1926_2A.png', './pieces_dataV3/spectrum\\gN_01_1927_2A.png', './pieces_dataV3/spectrum\\gN_01_1928_2A.png', './pieces_dataV3/spectrum\\gN_01_1929_2B.png', './pieces_dataV3/spectrum\\gN_01_1930_2B.png', './pieces_dataV3/spectrum\\gN_01_1931_2B.png', './pieces_dataV3/spectrum\\gN_01_1932_2B.png', './pieces_dataV3/spectrum\\gN_01_1933_2B.png', './pieces_dataV3/spectrum\\gN_01_1934_2B.png', './pieces_dataV3/spectrum\\gN_01_1935_2B.png', './pieces_dataV3/spectrum\\gN_01_1936_2B.png', './pieces_dataV3/spectrum\\gP_01_2043_3B.png', './pieces_dataV3/spectrum\\gP_01_2044_3B.png', './pieces_dataV3/spectrum\\gP_01_2045_3B.png', './pieces_dataV3/spectrum\\gP_01_2046_3B.png', './pieces_dataV3/spectrum\\gP_01_2047_3B.png', './pieces_dataV3/spectrum\\gP_01_2048_3B.png', './pieces_dataV3/spectrum\\gP_01_2049_3B.png', './pieces_dataV3/spectrum\\gP_01_2050_3B.png', './pieces_dataV3/spectrum\\gP_01_2051_3B.png', './pieces_dataV3/spectrum\\gP_01_2052_3B.png', './pieces_dataV3/spectrum\\gP_01_2053_3B.png', './pieces_dataV3/spectrum\\gQ_01_2173_2C.png', './pieces_dataV3/spectrum\\gQ_01_2174_2C.png', './pieces_dataV3/spectrum\\gQ_01_2175_2C.png', './pieces_dataV3/spectrum\\gQ_01_2176_2C.png', './pieces_dataV3/spectrum\\gQ_01_2177_2C.png', './pieces_dataV3/spectrum\\gQ_01_2178_2C.png', './pieces_dataV3/spectrum\\gQ_01_2179_2C.png', './pieces_dataV3/spectrum\\gQ_01_2180_2C.png', './pieces_dataV3/spectrum\\gQ_01_2181_2C.png', './pieces_dataV3/spectrum\\gR_01_2291_3A.png', './pieces_dataV3/spectrum\\gR_01_2292_3B.png', './pieces_dataV3/spectrum\\gR_01_2293_3B.png', './pieces_dataV3/spectrum\\gR_01_2294_3B.png', './pieces_dataV3/spectrum\\gR_01_2295_3B.png', './pieces_dataV3/spectrum\\gR_01_2296_3B.png', './pieces_dataV3/spectrum\\gR_01_2297_3C.png', './pieces_dataV3/spectrum\\gR_01_2298_3C.png', './pieces_dataV3/spectrum\\gR_01_2299_3C.png', './pieces_dataV3/spectrum\\gR_01_2300_3C.png', './pieces_dataV3/spectrum\\gR_01_2301_3C.png', './pieces_dataV3/spectrum\\gS_01_2394_1C.png', './pieces_dataV3/spectrum\\gS_01_2395_2A.png', './pieces_dataV3/spectrum\\gS_01_2396_2A.png', './pieces_dataV3/spectrum\\gS_01_2397_2A.png', './pieces_dataV3/spectrum\\gS_01_2398_2A.png', './pieces_dataV3/spectrum\\gS_01_2399_2A.png', './pieces_dataV3/spectrum\\gS_01_2400_2A.png', './pieces_dataV3/spectrum\\gB_01_855_2A.png', './pieces_dataV3/spectrum\\gC_01_938_2A.png', './pieces_dataV3/spectrum\\gC_01_939_2A.png', './pieces_dataV3/spectrum\\gC_01_940_2A.png', './pieces_dataV3/spectrum\\gC_01_941_2A.png', './pieces_dataV3/spectrum\\gC_01_942_2A.png', './pieces_dataV3/spectrum\\gC_01_943_2A.png', './pieces_dataV3/spectrum\\gC_01_944_2A.png', './pieces_dataV3/spectrum\\gC_01_945_2A.png', './pieces_dataV3/spectrum\\gC_01_946_2A.png', './pieces_dataV3/spectrum\\gC_01_947_2A.png', './pieces_dataV3/spectrum\\gC_01_948_2B.png', './pieces_dataV3/spectrum\\gG_01_1355_1C.png', './pieces_dataV3/spectrum\\gG_01_1356_1C.png', './pieces_dataV3/spectrum\\gG_01_1357_1C.png', './pieces_dataV3/spectrum\\gG_01_1358_1C.png', './pieces_dataV3/spectrum\\gG_01_1359_2A.png', './pieces_dataV3/spectrum\\gG_01_1360_2A.png', './pieces_dataV3/spectrum\\gG_01_1361_2A.png', './pieces_dataV3/spectrum\\gG_01_1362_2A.png', './pieces_dataV3/spectrum\\gG_01_1363_2A.png', './pieces_dataV3/spectrum\\gG_01_1364_2A.png', './pieces_dataV3/spectrum\\gH_01_1463_2A.png', './pieces_dataV3/spectrum\\gH_01_1464_2A.png', './pieces_dataV3/spectrum\\gH_01_1465_2A.png', './pieces_dataV3/spectrum\\gH_01_1466_2A.png', './pieces_dataV3/spectrum\\gH_01_1467_2A.png', './pieces_dataV3/spectrum\\gJ_01_1559_1C.png', './pieces_dataV3/spectrum\\gJ_01_1560_1C.png', './pieces_dataV3/spectrum\\gJ_01_1561_1C.png', './pieces_dataV3/spectrum\\gJ_01_1562_1C.png', './pieces_dataV3/spectrum\\gJ_01_1563_1C.png', './pieces_dataV3/spectrum\\gJ_01_1564_1C.png', './pieces_dataV3/spectrum\\gJ_01_1565_1C.png', './pieces_dataV3/spectrum\\gJ_01_1566_1C.png', './pieces_dataV3/spectrum\\gJ_01_1567_1C.png', './pieces_dataV3/spectrum\\gJ_01_1568_1C.png', './pieces_dataV3/spectrum\\gL_01_1745_2A.png', './pieces_dataV3/spectrum\\gL_01_1746_2A.png', './pieces_dataV3/spectrum\\gL_01_1747_2A.png', './pieces_dataV3/spectrum\\gL_01_1748_2A.png', './pieces_dataV3/spectrum\\gL_01_1749_2B.png', './pieces_dataV3/spectrum\\gL_01_1750_2B.png', './pieces_dataV3/spectrum\\gL_01_1751_2B.png', './pieces_dataV3/spectrum\\gL_01_1752_2B.png', './pieces_dataV3/spectrum\\gM_01_1835_2A.png', './pieces_dataV3/spectrum\\gM_01_1836_2A.png', './pieces_dataV3/spectrum\\gM_01_1837_2A.png', './pieces_dataV3/spectrum\\gM_01_1838_2B.png', './pieces_dataV3/spectrum\\gM_01_1839_2B.png', './pieces_dataV3/spectrum\\gM_01_1840_2B.png', './pieces_dataV3/spectrum\\gM_01_1841_2B.png', './pieces_dataV3/spectrum\\gM_01_1842_2B.png', './pieces_dataV3/spectrum\\gM_01_1843_2B.png', './pieces_dataV3/spectrum\\gM_01_1844_2B.png', './pieces_dataV3/spectrum\\gN_01_1937_2C.png', './pieces_dataV3/spectrum\\gN_01_1938_2C.png', './pieces_dataV3/spectrum\\gN_01_1939_2C.png', './pieces_dataV3/spectrum\\gN_01_1940_2C.png', './pieces_dataV3/spectrum\\gN_01_1941_2C.png', './pieces_dataV3/spectrum\\gN_01_1942_2C.png', './pieces_dataV3/spectrum\\gN_01_1943_2C.png', './pieces_dataV3/spectrum\\gN_01_1944_2C.png', './pieces_dataV3/spectrum\\gN_01_1945_3A.png', './pieces_dataV3/spectrum\\gN_01_1946_3A.png', './pieces_dataV3/spectrum\\gN_01_1947_3A.png', './pieces_dataV3/spectrum\\gP_01_2054_3B.png', './pieces_dataV3/spectrum\\gP_01_2055_3B.png', './pieces_dataV3/spectrum\\gP_01_2056_3B.png', './pieces_dataV3/spectrum\\gP_01_2057_3B.png', './pieces_dataV3/spectrum\\gP_01_2058_3B.png', './pieces_dataV3/spectrum\\gP_01_2059_3B.png', './pieces_dataV3/spectrum\\gP_01_2060_3B.png', './pieces_dataV3/spectrum\\gP_01_2061_3B.png', './pieces_dataV3/spectrum\\gP_01_2062_3B.png', './pieces_dataV3/spectrum\\gP_01_2063_3B.png', './pieces_dataV3/spectrum\\gP_01_2064_3B.png', './pieces_dataV3/spectrum\\gP_01_2065_3C.png', './pieces_dataV3/spectrum\\gP_01_2066_3C.png', './pieces_dataV3/spectrum\\gQ_01_2182_2C.png', './pieces_dataV3/spectrum\\gQ_01_2191_3B.png', './pieces_dataV3/spectrum\\gQ_01_2192_3B.png', './pieces_dataV3/spectrum\\gQ_01_2193_3B.png', './pieces_dataV3/spectrum\\gQ_01_2194_3B.png', './pieces_dataV3/spectrum\\gR_01_2302_3C.png', './pieces_dataV3/spectrum\\gR_01_2303_3C.png', './pieces_dataV3/spectrum\\gR_01_2304_4A.png', './pieces_dataV3/spectrum\\gR_01_2305_4A.png', './pieces_dataV3/spectrum\\gR_01_2306_4A.png', './pieces_dataV3/spectrum\\gR_01_2307_4A.png', './pieces_dataV3/spectrum\\gR_01_2308_4A.png', './pieces_dataV3/spectrum\\gR_01_2309_4B.png', './pieces_dataV3/spectrum\\gR_01_2310_4B.png', './pieces_dataV3/spectrum\\gR_01_2311_4B.png', './pieces_dataV3/spectrum\\gR_01_2312_4B.png', './pieces_dataV3/spectrum\\gC_01_949_2B.png', './pieces_dataV3/spectrum\\gC_01_950_2B.png', './pieces_dataV3/spectrum\\gC_01_951_2B.png', './pieces_dataV3/spectrum\\gC_01_952_2B.png', './pieces_dataV3/spectrum\\gC_01_953_2B.png', './pieces_dataV3/spectrum\\gC_01_954_2B.png', './pieces_dataV3/spectrum\\gC_01_955_2B.png', './pieces_dataV3/spectrum\\gC_01_956_2B.png', './pieces_dataV3/spectrum\\gC_01_957_2B.png', './pieces_dataV3/spectrum\\gE_01_1171_2C.png', './pieces_dataV3/spectrum\\gE_01_1172_2C.png', './pieces_dataV3/spectrum\\gE_01_1173_2C.png', './pieces_dataV3/spectrum\\gG_01_1365_2A.png', './pieces_dataV3/spectrum\\gG_01_1366_2A.png', './pieces_dataV3/spectrum\\gG_01_1367_2A.png', './pieces_dataV3/spectrum\\gG_01_1368_2B.png', './pieces_dataV3/spectrum\\gG_01_1369_2B.png', './pieces_dataV3/spectrum\\gG_01_1370_2B.png', './pieces_dataV3/spectrum\\gG_01_1371_2B.png', './pieces_dataV3/spectrum\\gG_01_1372_2B.png', './pieces_dataV3/spectrum\\gG_01_1373_2B.png', './pieces_dataV3/spectrum\\gG_01_1374_2B.png', './pieces_dataV3/spectrum\\gH_01_1468_2A.png', './pieces_dataV3/spectrum\\gH_01_1469_2A.png', './pieces_dataV3/spectrum\\gH_01_1470_2A.png', './pieces_dataV3/spectrum\\gH_01_1471_2A.png', './pieces_dataV3/spectrum\\gH_01_1472_2A.png', './pieces_dataV3/spectrum\\gH_01_1473_2A.png', './pieces_dataV3/spectrum\\gH_01_1474_2A.png', './pieces_dataV3/spectrum\\gH_01_1475_2A.png', './pieces_dataV3/spectrum\\gH_01_1476_2A.png', './pieces_dataV3/spectrum\\gH_01_1477_2A.png', './pieces_dataV3/spectrum\\gJ_01_1569_1C.png', './pieces_dataV3/spectrum\\gJ_01_1570_1C.png', './pieces_dataV3/spectrum\\gJ_01_1571_1C.png', './pieces_dataV3/spectrum\\gJ_01_1572_1C.png', './pieces_dataV3/spectrum\\gJ_01_1573_1C.png', './pieces_dataV3/spectrum\\gJ_01_1574_1C.png', './pieces_dataV3/spectrum\\gJ_01_1575_1C.png', './pieces_dataV3/spectrum\\gJ_01_1576_1C.png', './pieces_dataV3/spectrum\\gJ_01_1577_1C.png', './pieces_dataV3/spectrum\\gJ_01_1578_1C.png', './pieces_dataV3/spectrum\\gK_01_1666_2C.png', './pieces_dataV3/spectrum\\gK_01_1667_2C.png', './pieces_dataV3/spectrum\\gK_01_1668_2C.png', './pieces_dataV3/spectrum\\gK_01_1669_2C.png', './pieces_dataV3/spectrum\\gK_01_1670_2C.png', './pieces_dataV3/spectrum\\gK_01_1671_2C.png', './pieces_dataV3/spectrum\\gK_01_1672_2C.png', './pieces_dataV3/spectrum\\gK_01_1673_2C.png', './pieces_dataV3/spectrum\\gL_01_1753_2B.png', './pieces_dataV3/spectrum\\gL_01_1754_2B.png', './pieces_dataV3/spectrum\\gL_01_1755_2B.png', './pieces_dataV3/spectrum\\gL_01_1756_2B.png', './pieces_dataV3/spectrum\\gL_01_1757_2B.png', './pieces_dataV3/spectrum\\gL_01_1758_2B.png', './pieces_dataV3/spectrum\\gL_01_1759_2B.png', './pieces_dataV3/spectrum\\gM_01_1845_2B.png', './pieces_dataV3/spectrum\\gM_01_1846_2B.png', './pieces_dataV3/spectrum\\gM_01_1847_2B.png', './pieces_dataV3/spectrum\\gM_01_1848_2B.png', './pieces_dataV3/spectrum\\gM_01_1849_2B.png', './pieces_dataV3/spectrum\\gN_01_1948_3A.png', './pieces_dataV3/spectrum\\gN_01_1949_3A.png', './pieces_dataV3/spectrum\\gN_01_1950_3A.png', './pieces_dataV3/spectrum\\gN_01_1951_3A.png', './pieces_dataV3/spectrum\\gN_01_1952_3B.png', './pieces_dataV3/spectrum\\gN_01_1953_3B.png', './pieces_dataV3/spectrum\\gN_01_1954_3B.png', './pieces_dataV3/spectrum\\gN_01_1955_3B.png', './pieces_dataV3/spectrum\\gN_01_1956_3B.png', './pieces_dataV3/spectrum\\gN_01_1957_3B.png', './pieces_dataV3/spectrum\\gN_01_1958_3B.png', './pieces_dataV3/spectrum\\gP_01_2067_3C.png', './pieces_dataV3/spectrum\\gP_01_2068_3C.png', './pieces_dataV3/spectrum\\gP_01_2069_3C.png', './pieces_dataV3/spectrum\\gP_01_2070_3C.png', './pieces_dataV3/spectrum\\gP_01_2071_3C.png', './pieces_dataV3/spectrum\\gP_01_2072_3C.png', './pieces_dataV3/spectrum\\gP_01_2073_3C.png', './pieces_dataV3/spectrum\\gP_01_2074_3C.png', './pieces_dataV3/spectrum\\gP_01_2075_3C.png', './pieces_dataV3/spectrum\\gQ_01_2201_3C.png', './pieces_dataV3/spectrum\\gQ_01_2202_3C.png', './pieces_dataV3/spectrum\\gQ_01_2203_3C.png', './pieces_dataV3/spectrum\\gQ_01_2204_3C.png', './pieces_dataV3/spectrum\\gQ_01_2205_3C.png', './pieces_dataV3/spectrum\\gQ_01_2206_3C.png', './pieces_dataV3/spectrum\\gR_01_2313_4C.png', './pieces_dataV3/spectrum\\gR_01_2314_4C.png', './pieces_dataV3/spectrum\\gR_01_2315_4C.png', './pieces_dataV3/spectrum\\gR_01_2316_4C.png', './pieces_dataV3/spectrum\\gR_01_2317_4C.png', './pieces_dataV3/spectrum\\gR_01_2318_4C.png', './pieces_dataV3/spectrum\\gG_01_1375_2B.png', './pieces_dataV3/spectrum\\gG_01_1376_2C.png', './pieces_dataV3/spectrum\\gG_01_1377_2C.png', './pieces_dataV3/spectrum\\gG_01_1378_2C.png', './pieces_dataV3/spectrum\\gG_01_1379_2C.png', './pieces_dataV3/spectrum\\gG_01_1380_2C.png', './pieces_dataV3/spectrum\\gG_01_1381_2C.png', './pieces_dataV3/spectrum\\gG_01_1382_2C.png', './pieces_dataV3/spectrum\\gG_01_1383_2C.png', './pieces_dataV3/spectrum\\gG_01_1384_2C.png', './pieces_dataV3/spectrum\\gG_01_1385_3A.png', './pieces_dataV3/spectrum\\gH_01_1478_2A.png', './pieces_dataV3/spectrum\\gH_01_1479_2A.png', './pieces_dataV3/spectrum\\gH_01_1480_2A.png', './pieces_dataV3/spectrum\\gH_01_1481_2A.png', './pieces_dataV3/spectrum\\gH_01_1482_2A.png', './pieces_dataV3/spectrum\\gH_01_1483_2A.png', './pieces_dataV3/spectrum\\gJ_01_1579_2A.png', './pieces_dataV3/spectrum\\gJ_01_1580_2A.png', './pieces_dataV3/spectrum\\gJ_01_1581_2A.png', './pieces_dataV3/spectrum\\gJ_01_1582_2A.png', './pieces_dataV3/spectrum\\gJ_01_1583_2A.png', './pieces_dataV3/spectrum\\gJ_01_1584_2A.png', './pieces_dataV3/spectrum\\gJ_01_1585_2A.png', './pieces_dataV3/spectrum\\gJ_01_1586_2A.png', './pieces_dataV3/spectrum\\gJ_01_1587_2A.png', './pieces_dataV3/spectrum\\gJ_01_1588_2A.png', './pieces_dataV3/spectrum\\gK_01_1674_2C.png', './pieces_dataV3/spectrum\\gK_01_1675_2C.png', './pieces_dataV3/spectrum\\gK_01_1676_2C.png', './pieces_dataV3/spectrum\\gK_01_1677_2C.png', './pieces_dataV3/spectrum\\gN_01_1959_3B.png', './pieces_dataV3/spectrum\\gN_01_1960_3B.png', './pieces_dataV3/spectrum\\gN_01_1961_3B.png', './pieces_dataV3/spectrum\\gN_01_1962_3B.png', './pieces_dataV3/spectrum\\gN_01_1963_3B.png', './pieces_dataV3/spectrum\\gN_01_1964_3B.png', './pieces_dataV3/spectrum\\gN_01_1965_3B.png', './pieces_dataV3/spectrum\\gN_01_1966_3B.png', './pieces_dataV3/spectrum\\gN_01_1967_3B.png', './pieces_dataV3/spectrum\\gN_01_1968_3B.png', './pieces_dataV3/spectrum\\gN_01_1969_3B.png', './pieces_dataV3/spectrum\\gP_01_2079_4A.png', './pieces_dataV3/spectrum\\gP_01_2080_4A.png', './pieces_dataV3/spectrum\\gP_01_2081_4A.png', './pieces_dataV3/spectrum\\gP_01_2082_4A.png', './pieces_dataV3/spectrum\\gP_01_2083_4A.png', './pieces_dataV3/spectrum\\gQ_01_2207_3C.png', './pieces_dataV3/spectrum\\gQ_01_2208_3C.png', './pieces_dataV3/spectrum\\gQ_01_2211_4B.png', './pieces_dataV3/spectrum\\gQ_01_2212_4B.png', './pieces_dataV3/spectrum\\gQ_01_2213_4B.png', './pieces_dataV3/spectrum\\gQ_01_2216_5A.png', './pieces_dataV3/spectrum\\gQ_01_2217_5A.png', './pieces_dataV3/spectrum\\gQ_01_2218_5A.png', './pieces_dataV3/spectrum\\gQ_01_2219_5A.png', './pieces_dataV3/spectrum\\gR_01_2327_5C.png', './pieces_dataV3/spectrum\\gR_01_2328_5C.png', './pieces_dataV3/spectrum\\gR_01_2329_5C.png', './pieces_dataV3/spectrum\\gR_01_2330_5C.png', './pieces_dataV3/spectrum\\gR_01_2331_6A.png', './pieces_dataV3/spectrum\\gR_01_2332_6A.png', './pieces_dataV3/spectrum\\gR_01_2333_6A.png', './pieces_dataV3/spectrum\\gR_01_2334_6A.png', './pieces_dataV3/spectrum\\gS_01_2426_2C.png', './pieces_dataV3/spectrum\\gS_01_2427_2C.png', './pieces_dataV3/spectrum\\gS_01_2428_2C.png', './pieces_dataV3/spectrum\\gG_01_1386_3A.png', './pieces_dataV3/spectrum\\gG_01_1387_3A.png', './pieces_dataV3/spectrum\\gG_01_1388_3A.png', './pieces_dataV3/spectrum\\gG_01_1389_3A.png', './pieces_dataV3/spectrum\\gG_01_1390_3A.png', './pieces_dataV3/spectrum\\gG_01_1391_3A.png', './pieces_dataV3/spectrum\\gG_01_1392_3A.png', './pieces_dataV3/spectrum\\gG_01_1393_3A.png', './pieces_dataV3/spectrum\\gG_01_1394_3A.png', './pieces_dataV3/spectrum\\gG_01_1395_3A.png', './pieces_dataV3/spectrum\\gJ_01_1589_2A.png', './pieces_dataV3/spectrum\\gJ_01_1590_2B.png', './pieces_dataV3/spectrum\\gJ_01_1591_2B.png', './pieces_dataV3/spectrum\\gJ_01_1592_2B.png', './pieces_dataV3/spectrum\\gJ_01_1593_2B.png', './pieces_dataV3/spectrum\\gJ_01_1594_2B.png', './pieces_dataV3/spectrum\\gJ_01_1595_2B.png', './pieces_dataV3/spectrum\\gJ_01_1596_2B.png', './pieces_dataV3/spectrum\\gJ_01_1597_2B.png', './pieces_dataV3/spectrum\\gJ_01_1598_2B.png', './pieces_dataV3/spectrum\\gN_01_1970_3B.png', './pieces_dataV3/spectrum\\gN_01_1971_3B.png', './pieces_dataV3/spectrum\\gN_01_1972_3B.png', './pieces_dataV3/spectrum\\gN_01_1973_3B.png', './pieces_dataV3/spectrum\\gN_01_1974_3C.png', './pieces_dataV3/spectrum\\gN_01_1975_3C.png', './pieces_dataV3/spectrum\\gN_01_1976_3C.png', './pieces_dataV3/spectrum\\gN_01_1977_3C.png', './pieces_dataV3/spectrum\\gN_01_1978_3C.png', './pieces_dataV3/spectrum\\gN_01_1979_3C.png', './pieces_dataV3/spectrum\\gN_01_1980_3C.png', './pieces_dataV3/spectrum\\gP_01_2098_4C.png', './pieces_dataV3/spectrum\\gP_01_2099_4C.png', './pieces_dataV3/spectrum\\gP_01_2100_4C.png', './pieces_dataV3/spectrum\\gP_01_2101_4C.png', './pieces_dataV3/spectrum\\gP_01_2102_4C.png', './pieces_dataV3/spectrum\\gP_01_2103_4C.png', './pieces_dataV3/spectrum\\gQ_01_2220_5A.png', './pieces_dataV3/spectrum\\gQ_01_2221_5A.png', './pieces_dataV3/spectrum\\gQ_01_2222_5A.png', './pieces_dataV3/spectrum\\gQ_01_2223_5B.png', './pieces_dataV3/spectrum\\gQ_01_2224_5B.png', './pieces_dataV3/spectrum\\gQ_01_2225_5B.png', './pieces_dataV3/spectrum\\gQ_01_2226_5C.png', './pieces_dataV3/spectrum\\gQ_01_2227_5C.png', './pieces_dataV3/spectrum\\gQ_01_2228_5C.png', './pieces_dataV3/spectrum\\gQ_01_2229_5C.png', './pieces_dataV3/spectrum\\gQ_01_2230_5C.png', './pieces_dataV3/spectrum\\gQ_01_2231_5C.png', './pieces_dataV3/spectrum\\gQ_01_2232_5C.png', './pieces_dataV3/spectrum\\gR_01_2335_6A.png', './pieces_dataV3/spectrum\\gR_01_2336_6A.png', './pieces_dataV3/spectrum\\gR_01_2337_6A.png', './pieces_dataV3/spectrum\\gR_01_2344_6C.png', './pieces_dataV3/spectrum\\gR_01_2345_6C.png', './pieces_dataV3/spectrum\\gS_01_2429_2C.png', './pieces_dataV3/spectrum\\gS_01_2430_2C.png', './pieces_dataV3/spectrum\\gS_01_2431_2C.png', './pieces_dataV3/spectrum\\gS_01_2432_2C.png', './pieces_dataV3/spectrum\\gS_01_2433_2C.png', './pieces_dataV3/spectrum\\gS_01_2434_3A.png', './pieces_dataV3/spectrum\\gS_01_2435_3A.png', './pieces_dataV3/spectrum\\gS_01_2436_3A.png', './pieces_dataV3/spectrum\\gS_01_2437_3A.png', './pieces_dataV3/spectrum\\gC_01_986_3B.png', './pieces_dataV3/spectrum\\gC_01_987_3B.png', './pieces_dataV3/spectrum\\gC_01_988_3B.png', './pieces_dataV3/spectrum\\gD_01_1098_2C.png', './pieces_dataV3/spectrum\\gD_01_1099_2C.png', './pieces_dataV3/spectrum\\gD_01_1100_2C.png', './pieces_dataV3/spectrum\\gD_01_1101_2C.png', './pieces_dataV3/spectrum\\gD_01_1102_2C.png', './pieces_dataV3/spectrum\\gD_01_1103_2C.png', './pieces_dataV3/spectrum\\gD_01_1104_2C.png', './pieces_dataV3/spectrum\\gD_01_1105_2C.png', './pieces_dataV3/spectrum\\gD_01_1106_2C.png', './pieces_dataV3/spectrum\\gD_01_1107_2C.png', './pieces_dataV3/spectrum\\gG_01_1396_3B.png', './pieces_dataV3/spectrum\\gG_01_1397_3B.png', './pieces_dataV3/spectrum\\gG_01_1398_3B.png', './pieces_dataV3/spectrum\\gG_01_1399_3B.png', './pieces_dataV3/spectrum\\gG_01_1400_3B.png', './pieces_dataV3/spectrum\\gG_01_1401_3B.png', './pieces_dataV3/spectrum\\gG_01_1402_3B.png', './pieces_dataV3/spectrum\\gG_01_1403_3B.png', './pieces_dataV3/spectrum\\gG_01_1404_3B.png', './pieces_dataV3/spectrum\\gG_01_1405_3B.png', './pieces_dataV3/spectrum\\gJ_01_1601_2C.png', './pieces_dataV3/spectrum\\gJ_01_1602_2C.png', './pieces_dataV3/spectrum\\gJ_01_1603_2C.png', './pieces_dataV3/spectrum\\gJ_01_1604_2C.png', './pieces_dataV3/spectrum\\gJ_01_1605_2C.png', './pieces_dataV3/spectrum\\gJ_01_1606_2C.png', './pieces_dataV3/spectrum\\gJ_01_1607_2C.png', './pieces_dataV3/spectrum\\gJ_01_1608_2C.png', './pieces_dataV3/spectrum\\gM_01_1874_3B.png', './pieces_dataV3/spectrum\\gM_01_1875_3B.png', './pieces_dataV3/spectrum\\gM_01_1876_3B.png', './pieces_dataV3/spectrum\\gM_01_1877_3B.png', './pieces_dataV3/spectrum\\gM_01_1878_3B.png', './pieces_dataV3/spectrum\\gM_01_1879_3B.png', './pieces_dataV3/spectrum\\gM_01_1880_3B.png', './pieces_dataV3/spectrum\\gN_01_1981_3C.png', './pieces_dataV3/spectrum\\gN_01_1982_4A.png', './pieces_dataV3/spectrum\\gN_01_1983_4A.png', './pieces_dataV3/spectrum\\gN_01_1984_4A.png', './pieces_dataV3/spectrum\\gN_01_1985_4A.png', './pieces_dataV3/spectrum\\gN_01_1986_4A.png', './pieces_dataV3/spectrum\\gN_01_1987_4A.png', './pieces_dataV3/spectrum\\gN_01_1988_4A.png', './pieces_dataV3/spectrum\\gN_01_1989_4B.png', './pieces_dataV3/spectrum\\gN_01_1990_4B.png', './pieces_dataV3/spectrum\\gN_01_1991_4B.png', './pieces_dataV3/spectrum\\gP_01_2105_5A.png', './pieces_dataV3/spectrum\\gP_01_2106_5A.png', './pieces_dataV3/spectrum\\gP_01_2107_5A.png', './pieces_dataV3/spectrum\\gP_01_2108_5A.png', './pieces_dataV3/spectrum\\gP_01_2109_5A.png', './pieces_dataV3/spectrum\\gP_01_2110_5A.png', './pieces_dataV3/spectrum\\gP_01_2111_5B.png', './pieces_dataV3/spectrum\\gP_01_2112_5B.png', './pieces_dataV3/spectrum\\gP_01_2113_5B.png', './pieces_dataV3/spectrum\\gP_01_2114_5B.png', './pieces_dataV3/spectrum\\gP_01_2115_5B.png', './pieces_dataV3/spectrum\\gP_01_2116_5B.png', './pieces_dataV3/spectrum\\gP_01_2117_5B.png', './pieces_dataV3/spectrum\\gQ_01_2233_5C.png', './pieces_dataV3/spectrum\\gQ_01_2234_5C.png', './pieces_dataV3/spectrum\\gR_01_2346_6C.png', './pieces_dataV3/spectrum\\gR_01_2347_6C.png', './pieces_dataV3/spectrum\\gR_01_2348_6C.png', './pieces_dataV3/spectrum\\gS_01_2438_3A.png', './pieces_dataV3/spectrum\\gS_01_2439_3B.png', './pieces_dataV3/spectrum\\gS_01_2440_3B.png', './pieces_dataV3/spectrum\\gS_01_2441_3B.png', './pieces_dataV3/spectrum\\gS_01_2442_3B.png', './pieces_dataV3/spectrum\\gS_01_2443_3B.png', './pieces_dataV3/spectrum\\gS_01_2444_3B.png', './pieces_dataV3/spectrum\\gS_01_2445_3C.png', './pieces_dataV3/spectrum\\gS_01_2446_3C.png', './pieces_dataV3/spectrum\\gC_01_989_3B.png', './pieces_dataV3/spectrum\\gC_01_990_3B.png', './pieces_dataV3/spectrum\\gC_01_991_3B.png', './pieces_dataV3/spectrum\\gC_01_992_3B.png', './pieces_dataV3/spectrum\\gC_01_993_3B.png', './pieces_dataV3/spectrum\\gC_01_994_3B.png', './pieces_dataV3/spectrum\\gC_01_995_3B.png', './pieces_dataV3/spectrum\\gC_01_996_3B.png', './pieces_dataV3/spectrum\\gD_01_1108_2C.png', './pieces_dataV3/spectrum\\gD_01_1109_2C.png', './pieces_dataV3/spectrum\\gD_01_1110_2C.png', './pieces_dataV3/spectrum\\gD_01_1111_2C.png', './pieces_dataV3/spectrum\\gD_01_1112_2C.png', './pieces_dataV3/spectrum\\gD_01_1113_2C.png', './pieces_dataV3/spectrum\\gD_01_1114_2C.png', './pieces_dataV3/spectrum\\gD_01_1115_2C.png', './pieces_dataV3/spectrum\\gD_01_1116_2C.png', './pieces_dataV3/spectrum\\gD_01_1117_2C.png', './pieces_dataV3/spectrum\\gD_01_1118_2C.png', './pieces_dataV3/spectrum\\gF_01_1304_3C.png', './pieces_dataV3/spectrum\\gF_01_1305_3C.png', './pieces_dataV3/spectrum\\gF_01_1306_3C.png', './pieces_dataV3/spectrum\\gF_01_1307_3C.png', './pieces_dataV3/spectrum\\gF_01_1308_3C.png', './pieces_dataV3/spectrum\\gF_01_1309_3C.png', './pieces_dataV3/spectrum\\gF_01_1310_3C.png', './pieces_dataV3/spectrum\\gF_01_1311_3C.png', './pieces_dataV3/spectrum\\gF_01_1312_3C.png', './pieces_dataV3/spectrum\\gF_01_1313_3C.png', './pieces_dataV3/spectrum\\gG_01_1406_3B.png', './pieces_dataV3/spectrum\\gG_01_1407_3C.png', './pieces_dataV3/spectrum\\gG_01_1408_3C.png', './pieces_dataV3/spectrum\\gG_01_1409_3C.png', './pieces_dataV3/spectrum\\gG_01_1410_3C.png', './pieces_dataV3/spectrum\\gG_01_1411_3C.png', './pieces_dataV3/spectrum\\gG_01_1412_3C.png', './pieces_dataV3/spectrum\\gG_01_1413_3C.png', './pieces_dataV3/spectrum\\gG_01_1414_3C.png', './pieces_dataV3/spectrum\\gG_01_1415_3C.png', './pieces_dataV3/spectrum\\gG_01_1416_3C.png', './pieces_dataV3/spectrum\\gH_01_1508_2C.png', './pieces_dataV3/spectrum\\gH_01_1509_2C.png', './pieces_dataV3/spectrum\\gH_01_1510_2C.png', './pieces_dataV3/spectrum\\gH_01_1511_2C.png', './pieces_dataV3/spectrum\\gH_01_1512_2C.png', './pieces_dataV3/spectrum\\gH_01_1513_2C.png', './pieces_dataV3/spectrum\\gH_01_1514_2C.png', './pieces_dataV3/spectrum\\gH_01_1515_2C.png', './pieces_dataV3/spectrum\\gH_01_1516_2C.png', './pieces_dataV3/spectrum\\gH_01_1517_2C.png', './pieces_dataV3/spectrum\\gH_01_1518_2C.png', './pieces_dataV3/spectrum\\gJ_01_1609_2C.png', './pieces_dataV3/spectrum\\gJ_01_1610_2C.png', './pieces_dataV3/spectrum\\gJ_01_1611_2C.png', './pieces_dataV3/spectrum\\gJ_01_1612_2C.png', './pieces_dataV3/spectrum\\gJ_01_1613_2C.png', './pieces_dataV3/spectrum\\gJ_01_1614_2C.png', './pieces_dataV3/spectrum\\gJ_01_1615_2C.png', './pieces_dataV3/spectrum\\gJ_01_1616_2C.png', './pieces_dataV3/spectrum\\gJ_01_1617_2C.png', './pieces_dataV3/spectrum\\gJ_01_1618_2C.png', './pieces_dataV3/spectrum\\gJ_01_1619_2C.png', './pieces_dataV3/spectrum\\gN_01_1992_4B.png', './pieces_dataV3/spectrum\\gN_01_1993_4B.png', './pieces_dataV3/spectrum\\gN_01_1994_4B.png', './pieces_dataV3/spectrum\\gN_01_1995_4B.png', './pieces_dataV3/spectrum\\gP_01_2118_5B.png', './pieces_dataV3/spectrum\\gP_01_2119_5B.png', './pieces_dataV3/spectrum\\gP_01_2120_5B.png', './pieces_dataV3/spectrum\\gQ_01_2254_6C.png', './pieces_dataV3/spectrum\\gQ_01_2255_6C.png', './pieces_dataV3/spectrum\\gQ_01_2256_6C.png', './pieces_dataV3/spectrum\\gQ_01_2257_6C.png', './pieces_dataV3/spectrum\\gQ_01_2258_6C.png', './pieces_dataV3/spectrum\\gR_01_2364_8B.png', './pieces_dataV3/spectrum\\gR_01_2365_8B.png', './pieces_dataV3/spectrum\\gR_01_2366_8C.png', './pieces_dataV3/spectrum\\gR_01_2367_8C.png', './pieces_dataV3/spectrum\\gS_01_2447_3C.png', './pieces_dataV3/spectrum\\gS_01_2448_3C.png', './pieces_dataV3/spectrum\\gS_01_2449_3C.png', './pieces_dataV3/spectrum\\gS_01_2450_3C.png', './pieces_dataV3/spectrum\\gS_01_2451_3C.png', './pieces_dataV3/spectrum\\gS_01_2452_3C.png', './pieces_dataV3/spectrum\\gS_01_2453_3C.png', './pieces_dataV3/spectrum\\gS_01_2454_3C.png', './pieces_dataV3/spectrum\\gS_01_2455_3C.png'], ['./pieces_dataV3/spectrum\\gB_01_817_1A.png', './pieces_dataV3/spectrum\\gB_01_818_1A.png', './pieces_dataV3/spectrum\\gB_01_819_1A.png', './pieces_dataV3/spectrum\\gB_01_820_1A.png', './pieces_dataV3/spectrum\\gB_01_821_1A.png', './pieces_dataV3/spectrum\\gB_01_822_1A.png', './pieces_dataV3/spectrum\\gB_01_823_1A.png', './pieces_dataV3/spectrum\\gB_01_824_1A.png', './pieces_dataV3/spectrum\\gB_01_825_1A.png', './pieces_dataV3/spectrum\\gD_01_1015_1A.png', './pieces_dataV3/spectrum\\gD_01_1016_1A.png', './pieces_dataV3/spectrum\\gD_01_1017_1A.png', './pieces_dataV3/spectrum\\gD_01_1018_1A.png', './pieces_dataV3/spectrum\\gD_01_1019_1A.png', './pieces_dataV3/spectrum\\gD_01_1020_1A.png', './pieces_dataV3/spectrum\\gD_01_1021_1A.png', './pieces_dataV3/spectrum\\gD_01_1022_1A.png', './pieces_dataV3/spectrum\\gD_01_1023_1A.png', './pieces_dataV3/spectrum\\gD_01_1024_1A.png', './pieces_dataV3/spectrum\\gF_01_1211_1A.png', './pieces_dataV3/spectrum\\gF_01_1212_1A.png', './pieces_dataV3/spectrum\\gF_01_1213_1A.png', './pieces_dataV3/spectrum\\gF_01_1214_1A.png', './pieces_dataV3/spectrum\\gF_01_1215_1A.png', './pieces_dataV3/spectrum\\gF_01_1216_1A.png', './pieces_dataV3/spectrum\\gF_01_1217_1A.png', './pieces_dataV3/spectrum\\gF_01_1218_1A.png', './pieces_dataV3/spectrum\\gF_01_1219_1A.png', './pieces_dataV3/spectrum\\gK_01_1620_1A.png', './pieces_dataV3/spectrum\\gK_01_1621_1A.png', './pieces_dataV3/spectrum\\gK_01_1622_1A.png', './pieces_dataV3/spectrum\\gK_01_1623_1A.png', './pieces_dataV3/spectrum\\gK_01_1624_1A.png', './pieces_dataV3/spectrum\\gK_01_1625_1A.png', './pieces_dataV3/spectrum\\gK_01_1626_1A.png', './pieces_dataV3/spectrum\\gK_01_1627_1A.png', './pieces_dataV3/spectrum\\gN_01_1894_1A.png', './pieces_dataV3/spectrum\\gN_01_1895_1A.png', './pieces_dataV3/spectrum\\gN_01_1896_1A.png', './pieces_dataV3/spectrum\\gN_01_1897_1A.png', './pieces_dataV3/spectrum\\gN_01_1898_1A.png', './pieces_dataV3/spectrum\\gN_01_1899_1A.png', './pieces_dataV3/spectrum\\gP_01_2003_1A.png', './pieces_dataV3/spectrum\\gP_01_2004_1A.png', './pieces_dataV3/spectrum\\gP_01_2005_1A.png', './pieces_dataV3/spectrum\\gP_01_2006_1A.png', './pieces_dataV3/spectrum\\gP_01_2007_1A.png', './pieces_dataV3/spectrum\\gP_01_2008_1B.png', './pieces_dataV3/spectrum\\gP_01_2009_1B.png', './pieces_dataV3/spectrum\\gP_01_2010_1B.png', './pieces_dataV3/spectrum\\gP_01_2011_1B.png', './pieces_dataV3/spectrum\\gP_01_2012_1B.png', './pieces_dataV3/spectrum\\gP_01_2013_1B.png', './pieces_dataV3/spectrum\\gQ_01_2131_1A.png', './pieces_dataV3/spectrum\\gQ_01_2132_1A.png', './pieces_dataV3/spectrum\\gQ_01_2133_1A.png', './pieces_dataV3/spectrum\\gQ_01_2134_1A.png', './pieces_dataV3/spectrum\\gQ_01_2135_1A.png', './pieces_dataV3/spectrum\\gQ_01_2136_1A.png', './pieces_dataV3/spectrum\\gQ_01_2137_1A.png', './pieces_dataV3/spectrum\\gQ_01_2138_1A.png', './pieces_dataV3/spectrum\\gQ_01_2139_1A.png', './pieces_dataV3/spectrum\\gQ_01_2140_1A.png', './pieces_dataV3/spectrum\\gQ_01_2141_1A.png', './pieces_dataV3/spectrum\\gQ_01_2142_1A.png', './pieces_dataV3/spectrum\\gR_01_2259_1A.png', './pieces_dataV3/spectrum\\gR_01_2260_1A.png', './pieces_dataV3/spectrum\\gR_01_2261_1A.png', './pieces_dataV3/spectrum\\gR_01_2262_1A.png', './pieces_dataV3/spectrum\\gS_01_2368_1A.png', './pieces_dataV3/spectrum\\gS_01_2369_1A.png', './pieces_dataV3/spectrum\\gS_01_2370_1A.png', './pieces_dataV3/spectrum\\gS_01_2371_1A.png', './pieces_dataV3/spectrum\\gS_01_2372_1A.png', './pieces_dataV3/spectrum\\gS_01_2373_1A.png', './pieces_dataV3/spectrum\\gS_01_2374_1A.png', './pieces_dataV3/spectrum\\gS_01_2375_1A.png', './pieces_dataV3/spectrum\\gD_01_1025_1B.png', './pieces_dataV3/spectrum\\gD_01_1026_1B.png', './pieces_dataV3/spectrum\\gD_01_1027_1B.png', './pieces_dataV3/spectrum\\gD_01_1028_1B.png', './pieces_dataV3/spectrum\\gD_01_1029_1B.png', './pieces_dataV3/spectrum\\gD_01_1030_1B.png', './pieces_dataV3/spectrum\\gD_01_1031_1B.png', './pieces_dataV3/spectrum\\gD_01_1032_1B.png', './pieces_dataV3/spectrum\\gR_01_2273_2A.png', './pieces_dataV3/spectrum\\gR_01_2274_2A.png', './pieces_dataV3/spectrum\\gR_01_2275_2A.png', './pieces_dataV3/spectrum\\gR_01_2276_2B.png', './pieces_dataV3/spectrum\\gR_01_2277_2B.png', './pieces_dataV3/spectrum\\gR_01_2278_2B.png', './pieces_dataV3/spectrum\\gQ_01_2157_1C.png', './pieces_dataV3/spectrum\\gQ_01_2158_1C.png', './pieces_dataV3/spectrum\\gQ_01_2159_1C.png', './pieces_dataV3/spectrum\\gQ_01_2160_1C.png', './pieces_dataV3/spectrum\\gQ_01_2161_1C.png', './pieces_dataV3/spectrum\\gQ_01_2162_2A.png', './pieces_dataV3/spectrum\\gQ_01_2163_2A.png', './pieces_dataV3/spectrum\\gQ_01_2164_2A.png', './pieces_dataV3/spectrum\\gQ_01_2165_2A.png', './pieces_dataV3/spectrum\\gQ_01_2166_2A.png', './pieces_dataV3/spectrum\\gC_01_929_1C.png', './pieces_dataV3/spectrum\\gC_01_930_1C.png', './pieces_dataV3/spectrum\\gC_01_931_1C.png', './pieces_dataV3/spectrum\\gC_01_932_1C.png', './pieces_dataV3/spectrum\\gC_01_933_1C.png', './pieces_dataV3/spectrum\\gC_01_934_1C.png', './pieces_dataV3/spectrum\\gC_01_935_1C.png', './pieces_dataV3/spectrum\\gC_01_936_1C.png', './pieces_dataV3/spectrum\\gC_01_937_1C.png', './pieces_dataV3/spectrum\\gS_01_2395_2A.png', './pieces_dataV3/spectrum\\gS_01_2396_2A.png', './pieces_dataV3/spectrum\\gS_01_2397_2A.png', './pieces_dataV3/spectrum\\gS_01_2398_2A.png', './pieces_dataV3/spectrum\\gS_01_2399_2A.png', './pieces_dataV3/spectrum\\gS_01_2400_2A.png', './pieces_dataV3/spectrum\\gL_01_1745_2A.png', './pieces_dataV3/spectrum\\gL_01_1746_2A.png', './pieces_dataV3/spectrum\\gL_01_1747_2A.png', './pieces_dataV3/spectrum\\gL_01_1748_2A.png', './pieces_dataV3/spectrum\\gN_01_1937_2C.png', './pieces_dataV3/spectrum\\gN_01_1938_2C.png', './pieces_dataV3/spectrum\\gN_01_1939_2C.png', './pieces_dataV3/spectrum\\gN_01_1940_2C.png', './pieces_dataV3/spectrum\\gN_01_1941_2C.png', './pieces_dataV3/spectrum\\gN_01_1942_2C.png', './pieces_dataV3/spectrum\\gN_01_1943_2C.png', './pieces_dataV3/spectrum\\gN_01_1944_2C.png', './pieces_dataV3/spectrum\\gQ_01_2183_3A.png', './pieces_dataV3/spectrum\\gQ_01_2184_3A.png', './pieces_dataV3/spectrum\\gQ_01_2185_3A.png', './pieces_dataV3/spectrum\\gQ_01_2186_3A.png', './pieces_dataV3/spectrum\\gQ_01_2187_3A.png', './pieces_dataV3/spectrum\\gQ_01_2188_3A.png', './pieces_dataV3/spectrum\\gQ_01_2189_3A.png', './pieces_dataV3/spectrum\\gQ_01_2190_3A.png', './pieces_dataV3/spectrum\\gR_01_2304_4A.png', './pieces_dataV3/spectrum\\gR_01_2305_4A.png', './pieces_dataV3/spectrum\\gR_01_2306_4A.png', './pieces_dataV3/spectrum\\gR_01_2307_4A.png', './pieces_dataV3/spectrum\\gR_01_2308_4A.png', './pieces_dataV3/spectrum\\gR_01_2313_4C.png', './pieces_dataV3/spectrum\\gR_01_2314_4C.png', './pieces_dataV3/spectrum\\gR_01_2315_4C.png', './pieces_dataV3/spectrum\\gR_01_2316_4C.png', './pieces_dataV3/spectrum\\gR_01_2317_4C.png', './pieces_dataV3/spectrum\\gR_01_2318_4C.png', './pieces_dataV3/spectrum\\gG_01_1376_2C.png', './pieces_dataV3/spectrum\\gG_01_1377_2C.png', './pieces_dataV3/spectrum\\gG_01_1378_2C.png', './pieces_dataV3/spectrum\\gG_01_1379_2C.png', './pieces_dataV3/spectrum\\gG_01_1380_2C.png', './pieces_dataV3/spectrum\\gG_01_1381_2C.png', './pieces_dataV3/spectrum\\gG_01_1382_2C.png', './pieces_dataV3/spectrum\\gG_01_1383_2C.png', './pieces_dataV3/spectrum\\gG_01_1384_2C.png', './pieces_dataV3/spectrum\\gQ_01_2211_4B.png', './pieces_dataV3/spectrum\\gQ_01_2212_4B.png', './pieces_dataV3/spectrum\\gQ_01_2213_4B.png', './pieces_dataV3/spectrum\\gQ_01_2214_4C.png', './pieces_dataV3/spectrum\\gQ_01_2215_4C.png', './pieces_dataV3/spectrum\\gR_01_2327_5C.png', './pieces_dataV3/spectrum\\gR_01_2328_5C.png', './pieces_dataV3/spectrum\\gR_01_2329_5C.png', './pieces_dataV3/spectrum\\gR_01_2330_5C.png', './pieces_dataV3/spectrum\\gF_01_1284_3A.png', './pieces_dataV3/spectrum\\gF_01_1285_3A.png', './pieces_dataV3/spectrum\\gF_01_1286_3A.png', './pieces_dataV3/spectrum\\gF_01_1287_3A.png', './pieces_dataV3/spectrum\\gF_01_1288_3A.png', './pieces_dataV3/spectrum\\gF_01_1289_3A.png', './pieces_dataV3/spectrum\\gF_01_1290_3A.png', './pieces_dataV3/spectrum\\gF_01_1291_3A.png', './pieces_dataV3/spectrum\\gF_01_1292_3A.png', './pieces_dataV3/spectrum\\gM_01_1867_3A.png', './pieces_dataV3/spectrum\\gM_01_1868_3A.png', './pieces_dataV3/spectrum\\gM_01_1869_3A.png', './pieces_dataV3/spectrum\\gM_01_1870_3A.png', './pieces_dataV3/spectrum\\gM_01_1871_3A.png', './pieces_dataV3/spectrum\\gM_01_1872_3A.png', './pieces_dataV3/spectrum\\gM_01_1873_3A.png', './pieces_dataV3/spectrum\\gQ_01_2223_5B.png', './pieces_dataV3/spectrum\\gQ_01_2224_5B.png', './pieces_dataV3/spectrum\\gQ_01_2225_5B.png', './pieces_dataV3/spectrum\\gR_01_2338_6B.png', './pieces_dataV3/spectrum\\gR_01_2339_6B.png', './pieces_dataV3/spectrum\\gR_01_2340_6B.png', './pieces_dataV3/spectrum\\gR_01_2341_6B.png', './pieces_dataV3/spectrum\\gR_01_2342_6B.png', './pieces_dataV3/spectrum\\gR_01_2343_6B.png', './pieces_dataV3/spectrum\\gM_01_1874_3B.png', './pieces_dataV3/spectrum\\gM_01_1875_3B.png', './pieces_dataV3/spectrum\\gM_01_1876_3B.png', './pieces_dataV3/spectrum\\gM_01_1877_3B.png', './pieces_dataV3/spectrum\\gM_01_1878_3B.png', './pieces_dataV3/spectrum\\gM_01_1879_3B.png', './pieces_dataV3/spectrum\\gM_01_1880_3B.png', './pieces_dataV3/spectrum\\gN_01_1982_4A.png', './pieces_dataV3/spectrum\\gN_01_1983_4A.png', './pieces_dataV3/spectrum\\gN_01_1984_4A.png', './pieces_dataV3/spectrum\\gN_01_1985_4A.png', './pieces_dataV3/spectrum\\gN_01_1986_4A.png', './pieces_dataV3/spectrum\\gN_01_1987_4A.png', './pieces_dataV3/spectrum\\gN_01_1988_4A.png', './pieces_dataV3/spectrum\\gS_01_2439_3B.png', './pieces_dataV3/spectrum\\gS_01_2440_3B.png', './pieces_dataV3/spectrum\\gS_01_2441_3B.png', './pieces_dataV3/spectrum\\gS_01_2442_3B.png', './pieces_dataV3/spectrum\\gS_01_2443_3B.png', './pieces_dataV3/spectrum\\gS_01_2444_3B.png', './pieces_dataV3/spectrum\\gE_01_1202_3C.png', './pieces_dataV3/spectrum\\gE_01_1203_3C.png', './pieces_dataV3/spectrum\\gE_01_1204_3C.png', './pieces_dataV3/spectrum\\gE_01_1205_3C.png', './pieces_dataV3/spectrum\\gE_01_1206_3C.png', './pieces_dataV3/spectrum\\gE_01_1207_3C.png', './pieces_dataV3/spectrum\\gE_01_1208_3C.png', './pieces_dataV3/spectrum\\gE_01_1209_3C.png', './pieces_dataV3/spectrum\\gE_01_1210_3C.png', './pieces_dataV3/spectrum\\gF_01_1304_3C.png', './pieces_dataV3/spectrum\\gF_01_1305_3C.png', './pieces_dataV3/spectrum\\gF_01_1306_3C.png', './pieces_dataV3/spectrum\\gF_01_1307_3C.png', './pieces_dataV3/spectrum\\gF_01_1308_3C.png', './pieces_dataV3/spectrum\\gF_01_1309_3C.png', './pieces_dataV3/spectrum\\gF_01_1310_3C.png', './pieces_dataV3/spectrum\\gF_01_1311_3C.png', './pieces_dataV3/spectrum\\gF_01_1312_3C.png', './pieces_dataV3/spectrum\\gF_01_1313_3C.png', './pieces_dataV3/spectrum\\gG_01_1407_3C.png', './pieces_dataV3/spectrum\\gG_01_1408_3C.png', './pieces_dataV3/spectrum\\gG_01_1409_3C.png', './pieces_dataV3/spectrum\\gG_01_1410_3C.png', './pieces_dataV3/spectrum\\gG_01_1411_3C.png', './pieces_dataV3/spectrum\\gG_01_1412_3C.png', './pieces_dataV3/spectrum\\gG_01_1413_3C.png', './pieces_dataV3/spectrum\\gG_01_1414_3C.png', './pieces_dataV3/spectrum\\gG_01_1415_3C.png', './pieces_dataV3/spectrum\\gG_01_1416_3C.png', './pieces_dataV3/spectrum\\gN_01_1996_4C.png', './pieces_dataV3/spectrum\\gN_01_1997_4C.png', './pieces_dataV3/spectrum\\gN_01_1998_4C.png', './pieces_dataV3/spectrum\\gN_01_1999_4C.png', './pieces_dataV3/spectrum\\gN_01_2000_4C.png', './pieces_dataV3/spectrum\\gN_01_2001_4C.png', './pieces_dataV3/spectrum\\gN_01_2002_4C.png', './pieces_dataV3/spectrum\\gP_01_2121_5C.png', './pieces_dataV3/spectrum\\gP_01_2122_5C.png', './pieces_dataV3/spectrum\\gP_01_2123_5C.png', './pieces_dataV3/spectrum\\gP_01_2124_5C.png', './pieces_dataV3/spectrum\\gP_01_2125_5C.png', './pieces_dataV3/spectrum\\gP_01_2126_5C.png', './pieces_dataV3/spectrum\\gP_01_2127_5C.png', './pieces_dataV3/spectrum\\gP_01_2128_5C.png', './pieces_dataV3/spectrum\\gP_01_2129_5C.png', './pieces_dataV3/spectrum\\gP_01_2130_5C.png', './pieces_dataV3/spectrum\\gQ_01_2254_6C.png', './pieces_dataV3/spectrum\\gQ_01_2255_6C.png', './pieces_dataV3/spectrum\\gQ_01_2256_6C.png', './pieces_dataV3/spectrum\\gQ_01_2257_6C.png', './pieces_dataV3/spectrum\\gQ_01_2258_6C.png', './pieces_dataV3/spectrum\\gR_01_2358_7C.png', './pieces_dataV3/spectrum\\gR_01_2359_7C.png', './pieces_dataV3/spectrum\\gR_01_2360_7C.png', './pieces_dataV3/spectrum\\gR_01_2361_7C.png', './pieces_dataV3/spectrum\\gR_01_2366_8C.png', './pieces_dataV3/spectrum\\gR_01_2367_8C.png'], ['./pieces_dataV3/spectrum\\gB_01_817_1A.png', './pieces_dataV3/spectrum\\gB_01_818_1A.png', './pieces_dataV3/spectrum\\gB_01_819_1A.png', './pieces_dataV3/spectrum\\gB_01_820_1A.png', './pieces_dataV3/spectrum\\gB_01_821_1A.png', './pieces_dataV3/spectrum\\gB_01_822_1A.png', './pieces_dataV3/spectrum\\gB_01_823_1A.png', './pieces_dataV3/spectrum\\gB_01_824_1A.png', './pieces_dataV3/spectrum\\gB_01_825_1A.png', './pieces_dataV3/spectrum\\gD_01_1015_1A.png', './pieces_dataV3/spectrum\\gD_01_1016_1A.png', './pieces_dataV3/spectrum\\gD_01_1017_1A.png', './pieces_dataV3/spectrum\\gD_01_1018_1A.png', './pieces_dataV3/spectrum\\gD_01_1019_1A.png', './pieces_dataV3/spectrum\\gD_01_1020_1A.png', './pieces_dataV3/spectrum\\gD_01_1021_1A.png', './pieces_dataV3/spectrum\\gD_01_1022_1A.png', './pieces_dataV3/spectrum\\gD_01_1023_1A.png', './pieces_dataV3/spectrum\\gD_01_1024_1A.png', './pieces_dataV3/spectrum\\gF_01_1211_1A.png', './pieces_dataV3/spectrum\\gF_01_1212_1A.png', './pieces_dataV3/spectrum\\gF_01_1213_1A.png', './pieces_dataV3/spectrum\\gF_01_1214_1A.png', './pieces_dataV3/spectrum\\gF_01_1215_1A.png', './pieces_dataV3/spectrum\\gF_01_1216_1A.png', './pieces_dataV3/spectrum\\gF_01_1217_1A.png', './pieces_dataV3/spectrum\\gF_01_1218_1A.png', './pieces_dataV3/spectrum\\gF_01_1219_1A.png', './pieces_dataV3/spectrum\\gK_01_1620_1A.png', './pieces_dataV3/spectrum\\gK_01_1621_1A.png', './pieces_dataV3/spectrum\\gK_01_1622_1A.png', './pieces_dataV3/spectrum\\gK_01_1623_1A.png', './pieces_dataV3/spectrum\\gK_01_1624_1A.png', './pieces_dataV3/spectrum\\gK_01_1625_1A.png', './pieces_dataV3/spectrum\\gK_01_1626_1A.png', './pieces_dataV3/spectrum\\gK_01_1627_1A.png', './pieces_dataV3/spectrum\\gN_01_1894_1A.png', './pieces_dataV3/spectrum\\gN_01_1895_1A.png', './pieces_dataV3/spectrum\\gN_01_1896_1A.png', './pieces_dataV3/spectrum\\gN_01_1897_1A.png', './pieces_dataV3/spectrum\\gN_01_1898_1A.png', './pieces_dataV3/spectrum\\gN_01_1899_1A.png', './pieces_dataV3/spectrum\\gP_01_2003_1A.png', './pieces_dataV3/spectrum\\gP_01_2004_1A.png', './pieces_dataV3/spectrum\\gP_01_2005_1A.png', './pieces_dataV3/spectrum\\gP_01_2006_1A.png', './pieces_dataV3/spectrum\\gP_01_2007_1A.png', './pieces_dataV3/spectrum\\gP_01_2008_1B.png', './pieces_dataV3/spectrum\\gP_01_2009_1B.png', './pieces_dataV3/spectrum\\gP_01_2010_1B.png', './pieces_dataV3/spectrum\\gP_01_2011_1B.png', './pieces_dataV3/spectrum\\gP_01_2012_1B.png', './pieces_dataV3/spectrum\\gP_01_2013_1B.png', './pieces_dataV3/spectrum\\gR_01_2259_1A.png', './pieces_dataV3/spectrum\\gR_01_2260_1A.png', './pieces_dataV3/spectrum\\gR_01_2261_1A.png', './pieces_dataV3/spectrum\\gR_01_2262_1A.png', './pieces_dataV3/spectrum\\gS_01_2368_1A.png', './pieces_dataV3/spectrum\\gS_01_2369_1A.png', './pieces_dataV3/spectrum\\gS_01_2370_1A.png', './pieces_dataV3/spectrum\\gS_01_2371_1A.png', './pieces_dataV3/spectrum\\gS_01_2372_1A.png', './pieces_dataV3/spectrum\\gS_01_2373_1A.png', './pieces_dataV3/spectrum\\gS_01_2374_1A.png', './pieces_dataV3/spectrum\\gS_01_2375_1A.png', './pieces_dataV3/spectrum\\gR_01_2273_2A.png', './pieces_dataV3/spectrum\\gR_01_2274_2A.png', './pieces_dataV3/spectrum\\gR_01_2275_2A.png', './pieces_dataV3/spectrum\\gC_01_929_1C.png', './pieces_dataV3/spectrum\\gC_01_930_1C.png', './pieces_dataV3/spectrum\\gC_01_931_1C.png', './pieces_dataV3/spectrum\\gC_01_932_1C.png', './pieces_dataV3/spectrum\\gC_01_933_1C.png', './pieces_dataV3/spectrum\\gC_01_934_1C.png', './pieces_dataV3/spectrum\\gC_01_935_1C.png', './pieces_dataV3/spectrum\\gC_01_936_1C.png', './pieces_dataV3/spectrum\\gC_01_937_1C.png', './pieces_dataV3/spectrum\\gN_01_1929_2B.png', './pieces_dataV3/spectrum\\gN_01_1930_2B.png', './pieces_dataV3/spectrum\\gN_01_1931_2B.png', './pieces_dataV3/spectrum\\gN_01_1932_2B.png', './pieces_dataV3/spectrum\\gN_01_1933_2B.png', './pieces_dataV3/spectrum\\gN_01_1934_2B.png', './pieces_dataV3/spectrum\\gN_01_1935_2B.png', './pieces_dataV3/spectrum\\gN_01_1936_2B.png', './pieces_dataV3/spectrum\\gR_01_2292_3B.png', './pieces_dataV3/spectrum\\gR_01_2293_3B.png', './pieces_dataV3/spectrum\\gR_01_2294_3B.png', './pieces_dataV3/spectrum\\gR_01_2295_3B.png', './pieces_dataV3/spectrum\\gR_01_2296_3B.png', './pieces_dataV3/spectrum\\gS_01_2395_2A.png', './pieces_dataV3/spectrum\\gS_01_2396_2A.png', './pieces_dataV3/spectrum\\gS_01_2397_2A.png', './pieces_dataV3/spectrum\\gS_01_2398_2A.png', './pieces_dataV3/spectrum\\gS_01_2399_2A.png', './pieces_dataV3/spectrum\\gS_01_2400_2A.png', './pieces_dataV3/spectrum\\gL_01_1745_2A.png', './pieces_dataV3/spectrum\\gL_01_1746_2A.png', './pieces_dataV3/spectrum\\gL_01_1747_2A.png', './pieces_dataV3/spectrum\\gL_01_1748_2A.png', './pieces_dataV3/spectrum\\gN_01_1937_2C.png', './pieces_dataV3/spectrum\\gN_01_1938_2C.png', './pieces_dataV3/spectrum\\gN_01_1939_2C.png', './pieces_dataV3/spectrum\\gN_01_1940_2C.png', './pieces_dataV3/spectrum\\gN_01_1941_2C.png', './pieces_dataV3/spectrum\\gN_01_1942_2C.png', './pieces_dataV3/spectrum\\gN_01_1943_2C.png', './pieces_dataV3/spectrum\\gN_01_1944_2C.png', './pieces_dataV3/spectrum\\gQ_01_2183_3A.png', './pieces_dataV3/spectrum\\gQ_01_2184_3A.png', './pieces_dataV3/spectrum\\gQ_01_2185_3A.png', './pieces_dataV3/spectrum\\gQ_01_2186_3A.png', './pieces_dataV3/spectrum\\gQ_01_2187_3A.png', './pieces_dataV3/spectrum\\gQ_01_2188_3A.png', './pieces_dataV3/spectrum\\gQ_01_2189_3A.png', './pieces_dataV3/spectrum\\gQ_01_2190_3A.png', './pieces_dataV3/spectrum\\gR_01_2313_4C.png', './pieces_dataV3/spectrum\\gR_01_2314_4C.png', './pieces_dataV3/spectrum\\gR_01_2315_4C.png', './pieces_dataV3/spectrum\\gR_01_2316_4C.png', './pieces_dataV3/spectrum\\gR_01_2317_4C.png', './pieces_dataV3/spectrum\\gR_01_2318_4C.png', './pieces_dataV3/spectrum\\gF_01_1284_3A.png', './pieces_dataV3/spectrum\\gF_01_1285_3A.png', './pieces_dataV3/spectrum\\gF_01_1286_3A.png', './pieces_dataV3/spectrum\\gF_01_1287_3A.png', './pieces_dataV3/spectrum\\gF_01_1288_3A.png', './pieces_dataV3/spectrum\\gF_01_1289_3A.png', './pieces_dataV3/spectrum\\gF_01_1290_3A.png', './pieces_dataV3/spectrum\\gF_01_1291_3A.png', './pieces_dataV3/spectrum\\gF_01_1292_3A.png', './pieces_dataV3/spectrum\\gP_01_2098_4C.png', './pieces_dataV3/spectrum\\gP_01_2099_4C.png', './pieces_dataV3/spectrum\\gP_01_2100_4C.png', './pieces_dataV3/spectrum\\gP_01_2101_4C.png', './pieces_dataV3/spectrum\\gP_01_2102_4C.png', './pieces_dataV3/spectrum\\gP_01_2103_4C.png', './pieces_dataV3/spectrum\\gQ_01_2223_5B.png', './pieces_dataV3/spectrum\\gQ_01_2224_5B.png', './pieces_dataV3/spectrum\\gQ_01_2225_5B.png', './pieces_dataV3/spectrum\\gR_01_2338_6B.png', './pieces_dataV3/spectrum\\gR_01_2339_6B.png', './pieces_dataV3/spectrum\\gR_01_2340_6B.png', './pieces_dataV3/spectrum\\gR_01_2341_6B.png', './pieces_dataV3/spectrum\\gR_01_2342_6B.png', './pieces_dataV3/spectrum\\gR_01_2343_6B.png', './pieces_dataV3/spectrum\\gM_01_1874_3B.png', './pieces_dataV3/spectrum\\gM_01_1875_3B.png', './pieces_dataV3/spectrum\\gM_01_1876_3B.png', './pieces_dataV3/spectrum\\gM_01_1877_3B.png', './pieces_dataV3/spectrum\\gM_01_1878_3B.png', './pieces_dataV3/spectrum\\gM_01_1879_3B.png', './pieces_dataV3/spectrum\\gM_01_1880_3B.png', './pieces_dataV3/spectrum\\gN_01_1982_4A.png', './pieces_dataV3/spectrum\\gN_01_1983_4A.png', './pieces_dataV3/spectrum\\gN_01_1984_4A.png', './pieces_dataV3/spectrum\\gN_01_1985_4A.png', './pieces_dataV3/spectrum\\gN_01_1986_4A.png', './pieces_dataV3/spectrum\\gN_01_1987_4A.png', './pieces_dataV3/spectrum\\gN_01_1988_4A.png', './pieces_dataV3/spectrum\\gN_01_1996_4C.png', './pieces_dataV3/spectrum\\gN_01_1997_4C.png', './pieces_dataV3/spectrum\\gN_01_1998_4C.png', './pieces_dataV3/spectrum\\gN_01_1999_4C.png', './pieces_dataV3/spectrum\\gN_01_2000_4C.png', './pieces_dataV3/spectrum\\gN_01_2001_4C.png', './pieces_dataV3/spectrum\\gN_01_2002_4C.png', './pieces_dataV3/spectrum\\gQ_01_2254_6C.png', './pieces_dataV3/spectrum\\gQ_01_2255_6C.png', './pieces_dataV3/spectrum\\gQ_01_2256_6C.png', './pieces_dataV3/spectrum\\gQ_01_2257_6C.png', './pieces_dataV3/spectrum\\gQ_01_2258_6C.png', './pieces_dataV3/spectrum\\gR_01_2358_7C.png', './pieces_dataV3/spectrum\\gR_01_2359_7C.png', './pieces_dataV3/spectrum\\gR_01_2360_7C.png', './pieces_dataV3/spectrum\\gR_01_2361_7C.png', './pieces_dataV3/spectrum\\gR_01_2366_8C.png', './pieces_dataV3/spectrum\\gR_01_2367_8C.png'], ['./pieces_dataV3/spectrum\\gF_01_1211_1A.png', './pieces_dataV3/spectrum\\gF_01_1212_1A.png', './pieces_dataV3/spectrum\\gF_01_1213_1A.png', './pieces_dataV3/spectrum\\gF_01_1214_1A.png', './pieces_dataV3/spectrum\\gF_01_1215_1A.png', './pieces_dataV3/spectrum\\gF_01_1216_1A.png', './pieces_dataV3/spectrum\\gF_01_1217_1A.png', './pieces_dataV3/spectrum\\gF_01_1218_1A.png', './pieces_dataV3/spectrum\\gF_01_1219_1A.png', './pieces_dataV3/spectrum\\gK_01_1620_1A.png', './pieces_dataV3/spectrum\\gK_01_1621_1A.png', './pieces_dataV3/spectrum\\gK_01_1622_1A.png', './pieces_dataV3/spectrum\\gK_01_1623_1A.png', './pieces_dataV3/spectrum\\gK_01_1624_1A.png', './pieces_dataV3/spectrum\\gK_01_1625_1A.png', './pieces_dataV3/spectrum\\gK_01_1626_1A.png', './pieces_dataV3/spectrum\\gK_01_1627_1A.png', './pieces_dataV3/spectrum\\gM_01_1796_1A.png', './pieces_dataV3/spectrum\\gM_01_1797_1A.png', './pieces_dataV3/spectrum\\gM_01_1798_1A.png', './pieces_dataV3/spectrum\\gM_01_1799_1A.png', './pieces_dataV3/spectrum\\gM_01_1800_1A.png', './pieces_dataV3/spectrum\\gM_01_1801_1A.png', './pieces_dataV3/spectrum\\gM_01_1802_1A.png', './pieces_dataV3/spectrum\\gM_01_1803_1A.png', './pieces_dataV3/spectrum\\gM_01_1804_1A.png', './pieces_dataV3/spectrum\\gN_01_1894_1A.png', './pieces_dataV3/spectrum\\gN_01_1895_1A.png', './pieces_dataV3/spectrum\\gN_01_1896_1A.png', './pieces_dataV3/spectrum\\gN_01_1897_1A.png', './pieces_dataV3/spectrum\\gN_01_1898_1A.png', './pieces_dataV3/spectrum\\gN_01_1899_1A.png', './pieces_dataV3/spectrum\\gP_01_2003_1A.png', './pieces_dataV3/spectrum\\gP_01_2004_1A.png', './pieces_dataV3/spectrum\\gP_01_2005_1A.png', './pieces_dataV3/spectrum\\gP_01_2006_1A.png', './pieces_dataV3/spectrum\\gP_01_2007_1A.png', './pieces_dataV3/spectrum\\gP_01_2008_1B.png', './pieces_dataV3/spectrum\\gP_01_2009_1B.png', './pieces_dataV3/spectrum\\gP_01_2010_1B.png', './pieces_dataV3/spectrum\\gP_01_2011_1B.png', './pieces_dataV3/spectrum\\gP_01_2012_1B.png', './pieces_dataV3/spectrum\\gP_01_2013_1B.png', './pieces_dataV3/spectrum\\gR_01_2259_1A.png', './pieces_dataV3/spectrum\\gR_01_2260_1A.png', './pieces_dataV3/spectrum\\gR_01_2261_1A.png', './pieces_dataV3/spectrum\\gR_01_2262_1A.png', './pieces_dataV3/spectrum\\gR_01_2263_1B.png', './pieces_dataV3/spectrum\\gR_01_2264_1B.png', './pieces_dataV3/spectrum\\gR_01_2265_1B.png', './pieces_dataV3/spectrum\\gR_01_2266_1B.png', './pieces_dataV3/spectrum\\gS_01_2368_1A.png', './pieces_dataV3/spectrum\\gS_01_2369_1A.png', './pieces_dataV3/spectrum\\gS_01_2370_1A.png', './pieces_dataV3/spectrum\\gS_01_2371_1A.png', './pieces_dataV3/spectrum\\gS_01_2372_1A.png', './pieces_dataV3/spectrum\\gS_01_2373_1A.png', './pieces_dataV3/spectrum\\gS_01_2374_1A.png', './pieces_dataV3/spectrum\\gS_01_2375_1A.png', './pieces_dataV3/spectrum\\gF_01_1229_1C.png', './pieces_dataV3/spectrum\\gF_01_1230_1C.png', './pieces_dataV3/spectrum\\gP_01_2015_1C.png', './pieces_dataV3/spectrum\\gP_01_2016_1C.png', './pieces_dataV3/spectrum\\gP_01_2017_1C.png', './pieces_dataV3/spectrum\\gP_01_2018_1C.png', './pieces_dataV3/spectrum\\gP_01_2019_1C.png', './pieces_dataV3/spectrum\\gP_01_2020_1C.png', './pieces_dataV3/spectrum\\gP_01_2021_1C.png', './pieces_dataV3/spectrum\\gP_01_2022_1C.png', './pieces_dataV3/spectrum\\gP_01_2027_2B.png', './pieces_dataV3/spectrum\\gR_01_2276_2B.png', './pieces_dataV3/spectrum\\gR_01_2277_2B.png', './pieces_dataV3/spectrum\\gR_01_2278_2B.png', './pieces_dataV3/spectrum\\gD_01_1035_1C.png', './pieces_dataV3/spectrum\\gD_01_1036_1C.png', './pieces_dataV3/spectrum\\gD_01_1037_1C.png', './pieces_dataV3/spectrum\\gD_01_1038_1C.png', './pieces_dataV3/spectrum\\gD_01_1039_1C.png', './pieces_dataV3/spectrum\\gD_01_1040_1C.png', './pieces_dataV3/spectrum\\gD_01_1041_1C.png', './pieces_dataV3/spectrum\\gD_01_1042_1C.png', './pieces_dataV3/spectrum\\gD_01_1043_1C.png', './pieces_dataV3/spectrum\\gD_01_1044_1C.png', './pieces_dataV3/spectrum\\gF_01_1231_1C.png', './pieces_dataV3/spectrum\\gF_01_1232_1C.png', './pieces_dataV3/spectrum\\gF_01_1233_1C.png', './pieces_dataV3/spectrum\\gF_01_1234_1C.png', './pieces_dataV3/spectrum\\gF_01_1235_1C.png', './pieces_dataV3/spectrum\\gF_01_1236_1C.png', './pieces_dataV3/spectrum\\gF_01_1237_1C.png', './pieces_dataV3/spectrum\\gF_01_1238_1C.png', './pieces_dataV3/spectrum\\gM_01_1816_1C.png', './pieces_dataV3/spectrum\\gM_01_1817_1C.png', './pieces_dataV3/spectrum\\gM_01_1818_1C.png', './pieces_dataV3/spectrum\\gM_01_1819_1C.png', './pieces_dataV3/spectrum\\gM_01_1820_1C.png', './pieces_dataV3/spectrum\\gM_01_1821_1C.png', './pieces_dataV3/spectrum\\gM_01_1822_1C.png', './pieces_dataV3/spectrum\\gM_01_1823_1C.png', './pieces_dataV3/spectrum\\gM_01_1824_1C.png', './pieces_dataV3/spectrum\\gP_01_2028_2B.png', './pieces_dataV3/spectrum\\gB_01_846_2A.png', './pieces_dataV3/spectrum\\gB_01_847_2A.png', './pieces_dataV3/spectrum\\gB_01_848_2A.png', './pieces_dataV3/spectrum\\gB_01_849_2A.png', './pieces_dataV3/spectrum\\gB_01_850_2A.png', './pieces_dataV3/spectrum\\gB_01_851_2A.png', './pieces_dataV3/spectrum\\gB_01_852_2A.png', './pieces_dataV3/spectrum\\gB_01_853_2A.png', './pieces_dataV3/spectrum\\gB_01_854_2A.png', './pieces_dataV3/spectrum\\gC_01_929_1C.png', './pieces_dataV3/spectrum\\gC_01_930_1C.png', './pieces_dataV3/spectrum\\gC_01_931_1C.png', './pieces_dataV3/spectrum\\gC_01_932_1C.png', './pieces_dataV3/spectrum\\gC_01_933_1C.png', './pieces_dataV3/spectrum\\gC_01_934_1C.png', './pieces_dataV3/spectrum\\gC_01_935_1C.png', './pieces_dataV3/spectrum\\gC_01_936_1C.png', './pieces_dataV3/spectrum\\gC_01_937_1C.png', './pieces_dataV3/spectrum\\gH_01_1447_1C.png', './pieces_dataV3/spectrum\\gH_01_1448_1C.png', './pieces_dataV3/spectrum\\gH_01_1449_1C.png', './pieces_dataV3/spectrum\\gH_01_1450_1C.png', './pieces_dataV3/spectrum\\gH_01_1451_1C.png', './pieces_dataV3/spectrum\\gH_01_1452_1C.png', './pieces_dataV3/spectrum\\gH_01_1453_1C.png', './pieces_dataV3/spectrum\\gH_01_1454_1C.png', './pieces_dataV3/spectrum\\gH_01_1455_1C.png', './pieces_dataV3/spectrum\\gH_01_1456_1C.png', './pieces_dataV3/spectrum\\gN_01_1929_2B.png', './pieces_dataV3/spectrum\\gN_01_1930_2B.png', './pieces_dataV3/spectrum\\gN_01_1931_2B.png', './pieces_dataV3/spectrum\\gN_01_1932_2B.png', './pieces_dataV3/spectrum\\gN_01_1933_2B.png', './pieces_dataV3/spectrum\\gN_01_1934_2B.png', './pieces_dataV3/spectrum\\gN_01_1935_2B.png', './pieces_dataV3/spectrum\\gN_01_1936_2B.png', './pieces_dataV3/spectrum\\gR_01_2292_3B.png', './pieces_dataV3/spectrum\\gR_01_2293_3B.png', './pieces_dataV3/spectrum\\gR_01_2294_3B.png', './pieces_dataV3/spectrum\\gR_01_2295_3B.png', './pieces_dataV3/spectrum\\gR_01_2296_3B.png', './pieces_dataV3/spectrum\\gC_01_948_2B.png', './pieces_dataV3/spectrum\\gL_01_1745_2A.png', './pieces_dataV3/spectrum\\gL_01_1746_2A.png', './pieces_dataV3/spectrum\\gL_01_1747_2A.png', './pieces_dataV3/spectrum\\gL_01_1748_2A.png', './pieces_dataV3/spectrum\\gN_01_1937_2C.png', './pieces_dataV3/spectrum\\gN_01_1938_2C.png', './pieces_dataV3/spectrum\\gN_01_1939_2C.png', './pieces_dataV3/spectrum\\gN_01_1940_2C.png', './pieces_dataV3/spectrum\\gN_01_1941_2C.png', './pieces_dataV3/spectrum\\gN_01_1942_2C.png', './pieces_dataV3/spectrum\\gN_01_1943_2C.png', './pieces_dataV3/spectrum\\gN_01_1944_2C.png', './pieces_dataV3/spectrum\\gQ_01_2183_3A.png', './pieces_dataV3/spectrum\\gQ_01_2184_3A.png', './pieces_dataV3/spectrum\\gQ_01_2185_3A.png', './pieces_dataV3/spectrum\\gQ_01_2186_3A.png', './pieces_dataV3/spectrum\\gQ_01_2187_3A.png', './pieces_dataV3/spectrum\\gQ_01_2188_3A.png', './pieces_dataV3/spectrum\\gQ_01_2189_3A.png', './pieces_dataV3/spectrum\\gQ_01_2190_3A.png', './pieces_dataV3/spectrum\\gC_01_949_2B.png', './pieces_dataV3/spectrum\\gC_01_950_2B.png', './pieces_dataV3/spectrum\\gC_01_951_2B.png', './pieces_dataV3/spectrum\\gC_01_952_2B.png', './pieces_dataV3/spectrum\\gC_01_953_2B.png', './pieces_dataV3/spectrum\\gC_01_954_2B.png', './pieces_dataV3/spectrum\\gC_01_955_2B.png', './pieces_dataV3/spectrum\\gC_01_956_2B.png', './pieces_dataV3/spectrum\\gC_01_957_2B.png', './pieces_dataV3/spectrum\\gG_01_1368_2B.png', './pieces_dataV3/spectrum\\gG_01_1369_2B.png', './pieces_dataV3/spectrum\\gG_01_1370_2B.png', './pieces_dataV3/spectrum\\gG_01_1371_2B.png', './pieces_dataV3/spectrum\\gG_01_1372_2B.png', './pieces_dataV3/spectrum\\gG_01_1373_2B.png', './pieces_dataV3/spectrum\\gG_01_1374_2B.png', './pieces_dataV3/spectrum\\gK_01_1666_2C.png', './pieces_dataV3/spectrum\\gK_01_1667_2C.png', './pieces_dataV3/spectrum\\gK_01_1668_2C.png', './pieces_dataV3/spectrum\\gK_01_1669_2C.png', './pieces_dataV3/spectrum\\gK_01_1670_2C.png', './pieces_dataV3/spectrum\\gK_01_1671_2C.png', './pieces_dataV3/spectrum\\gK_01_1672_2C.png', './pieces_dataV3/spectrum\\gK_01_1673_2C.png', './pieces_dataV3/spectrum\\gR_01_2313_4C.png', './pieces_dataV3/spectrum\\gR_01_2314_4C.png', './pieces_dataV3/spectrum\\gR_01_2315_4C.png', './pieces_dataV3/spectrum\\gR_01_2316_4C.png', './pieces_dataV3/spectrum\\gR_01_2317_4C.png', './pieces_dataV3/spectrum\\gR_01_2318_4C.png', './pieces_dataV3/spectrum\\gE_01_1174_2C.png', './pieces_dataV3/spectrum\\gE_01_1175_2C.png', './pieces_dataV3/spectrum\\gE_01_1176_2C.png', './pieces_dataV3/spectrum\\gE_01_1177_2C.png', './pieces_dataV3/spectrum\\gE_01_1178_2C.png', './pieces_dataV3/spectrum\\gE_01_1179_2C.png', './pieces_dataV3/spectrum\\gE_01_1180_2C.png', './pieces_dataV3/spectrum\\gE_01_1181_2C.png', './pieces_dataV3/spectrum\\gG_01_1375_2B.png', './pieces_dataV3/spectrum\\gQ_01_2214_4C.png', './pieces_dataV3/spectrum\\gQ_01_2215_4C.png', './pieces_dataV3/spectrum\\gQ_01_2216_5A.png', './pieces_dataV3/spectrum\\gQ_01_2217_5A.png', './pieces_dataV3/spectrum\\gQ_01_2218_5A.png', './pieces_dataV3/spectrum\\gQ_01_2219_5A.png', './pieces_dataV3/spectrum\\gJ_01_1590_2B.png', './pieces_dataV3/spectrum\\gJ_01_1591_2B.png', './pieces_dataV3/spectrum\\gJ_01_1592_2B.png', './pieces_dataV3/spectrum\\gJ_01_1593_2B.png', './pieces_dataV3/spectrum\\gJ_01_1594_2B.png', './pieces_dataV3/spectrum\\gJ_01_1595_2B.png', './pieces_dataV3/spectrum\\gJ_01_1596_2B.png', './pieces_dataV3/spectrum\\gJ_01_1597_2B.png', './pieces_dataV3/spectrum\\gJ_01_1598_2B.png', './pieces_dataV3/spectrum\\gM_01_1867_3A.png', './pieces_dataV3/spectrum\\gM_01_1868_3A.png', './pieces_dataV3/spectrum\\gM_01_1869_3A.png', './pieces_dataV3/spectrum\\gM_01_1870_3A.png', './pieces_dataV3/spectrum\\gM_01_1871_3A.png', './pieces_dataV3/spectrum\\gM_01_1872_3A.png', './pieces_dataV3/spectrum\\gM_01_1873_3A.png', './pieces_dataV3/spectrum\\gP_01_2098_4C.png', './pieces_dataV3/spectrum\\gP_01_2099_4C.png', './pieces_dataV3/spectrum\\gP_01_2100_4C.png', './pieces_dataV3/spectrum\\gP_01_2101_4C.png', './pieces_dataV3/spectrum\\gP_01_2102_4C.png', './pieces_dataV3/spectrum\\gP_01_2103_4C.png', './pieces_dataV3/spectrum\\gQ_01_2220_5A.png', './pieces_dataV3/spectrum\\gQ_01_2221_5A.png', './pieces_dataV3/spectrum\\gQ_01_2222_5A.png', './pieces_dataV3/spectrum\\gQ_01_2223_5B.png', './pieces_dataV3/spectrum\\gQ_01_2224_5B.png', './pieces_dataV3/spectrum\\gQ_01_2225_5B.png', './pieces_dataV3/spectrum\\gR_01_2338_6B.png', './pieces_dataV3/spectrum\\gR_01_2339_6B.png', './pieces_dataV3/spectrum\\gR_01_2340_6B.png', './pieces_dataV3/spectrum\\gR_01_2341_6B.png', './pieces_dataV3/spectrum\\gR_01_2342_6B.png', './pieces_dataV3/spectrum\\gR_01_2343_6B.png', './pieces_dataV3/spectrum\\gK_01_1698_3C.png', './pieces_dataV3/spectrum\\gK_01_1699_3C.png', './pieces_dataV3/spectrum\\gK_01_1700_3C.png', './pieces_dataV3/spectrum\\gM_01_1874_3B.png', './pieces_dataV3/spectrum\\gM_01_1875_3B.png', './pieces_dataV3/spectrum\\gM_01_1876_3B.png', './pieces_dataV3/spectrum\\gM_01_1877_3B.png', './pieces_dataV3/spectrum\\gM_01_1878_3B.png', './pieces_dataV3/spectrum\\gM_01_1879_3B.png', './pieces_dataV3/spectrum\\gM_01_1880_3B.png', './pieces_dataV3/spectrum\\gS_01_2439_3B.png', './pieces_dataV3/spectrum\\gS_01_2440_3B.png', './pieces_dataV3/spectrum\\gS_01_2441_3B.png', './pieces_dataV3/spectrum\\gS_01_2442_3B.png', './pieces_dataV3/spectrum\\gS_01_2443_3B.png', './pieces_dataV3/spectrum\\gS_01_2444_3B.png', './pieces_dataV3/spectrum\\gF_01_1304_3C.png', './pieces_dataV3/spectrum\\gF_01_1305_3C.png', './pieces_dataV3/spectrum\\gF_01_1306_3C.png', './pieces_dataV3/spectrum\\gF_01_1307_3C.png', './pieces_dataV3/spectrum\\gF_01_1308_3C.png', './pieces_dataV3/spectrum\\gF_01_1309_3C.png', './pieces_dataV3/spectrum\\gF_01_1310_3C.png', './pieces_dataV3/spectrum\\gF_01_1311_3C.png', './pieces_dataV3/spectrum\\gF_01_1312_3C.png', './pieces_dataV3/spectrum\\gF_01_1313_3C.png', './pieces_dataV3/spectrum\\gK_01_1701_3C.png', './pieces_dataV3/spectrum\\gK_01_1702_3C.png', './pieces_dataV3/spectrum\\gK_01_1703_3C.png', './pieces_dataV3/spectrum\\gK_01_1704_3C.png', './pieces_dataV3/spectrum\\gK_01_1705_3C.png', './pieces_dataV3/spectrum\\gK_01_1706_3C.png', './pieces_dataV3/spectrum\\gK_01_1707_3C.png', './pieces_dataV3/spectrum\\gK_01_1708_3C.png', './pieces_dataV3/spectrum\\gK_01_1709_3C.png', './pieces_dataV3/spectrum\\gN_01_1996_4C.png', './pieces_dataV3/spectrum\\gN_01_1997_4C.png', './pieces_dataV3/spectrum\\gN_01_1998_4C.png', './pieces_dataV3/spectrum\\gN_01_1999_4C.png', './pieces_dataV3/spectrum\\gN_01_2000_4C.png', './pieces_dataV3/spectrum\\gN_01_2001_4C.png', './pieces_dataV3/spectrum\\gN_01_2002_4C.png', './pieces_dataV3/spectrum\\gR_01_2366_8C.png', './pieces_dataV3/spectrum\\gR_01_2367_8C.png'], ['./pieces_dataV3/spectrum\\gB_01_817_1A.png', './pieces_dataV3/spectrum\\gB_01_818_1A.png', './pieces_dataV3/spectrum\\gB_01_819_1A.png', './pieces_dataV3/spectrum\\gB_01_820_1A.png', './pieces_dataV3/spectrum\\gB_01_821_1A.png', './pieces_dataV3/spectrum\\gB_01_822_1A.png', './pieces_dataV3/spectrum\\gB_01_823_1A.png', './pieces_dataV3/spectrum\\gB_01_824_1A.png', './pieces_dataV3/spectrum\\gB_01_825_1A.png', './pieces_dataV3/spectrum\\gM_01_1796_1A.png', './pieces_dataV3/spectrum\\gM_01_1797_1A.png', './pieces_dataV3/spectrum\\gM_01_1798_1A.png', './pieces_dataV3/spectrum\\gM_01_1799_1A.png', './pieces_dataV3/spectrum\\gM_01_1800_1A.png', './pieces_dataV3/spectrum\\gM_01_1801_1A.png', './pieces_dataV3/spectrum\\gM_01_1802_1A.png', './pieces_dataV3/spectrum\\gM_01_1803_1A.png', './pieces_dataV3/spectrum\\gM_01_1804_1A.png', './pieces_dataV3/spectrum\\gN_01_1894_1A.png', './pieces_dataV3/spectrum\\gN_01_1895_1A.png', './pieces_dataV3/spectrum\\gN_01_1896_1A.png', './pieces_dataV3/spectrum\\gN_01_1897_1A.png', './pieces_dataV3/spectrum\\gN_01_1898_1A.png', './pieces_dataV3/spectrum\\gN_01_1899_1A.png', './pieces_dataV3/spectrum\\gR_01_2259_1A.png', './pieces_dataV3/spectrum\\gR_01_2260_1A.png', './pieces_dataV3/spectrum\\gR_01_2261_1A.png', './pieces_dataV3/spectrum\\gR_01_2262_1A.png', './pieces_dataV3/spectrum\\gS_01_2368_1A.png', './pieces_dataV3/spectrum\\gS_01_2369_1A.png', './pieces_dataV3/spectrum\\gS_01_2370_1A.png', './pieces_dataV3/spectrum\\gS_01_2371_1A.png', './pieces_dataV3/spectrum\\gS_01_2372_1A.png', './pieces_dataV3/spectrum\\gS_01_2373_1A.png', './pieces_dataV3/spectrum\\gS_01_2374_1A.png', './pieces_dataV3/spectrum\\gS_01_2375_1A.png', './pieces_dataV3/spectrum\\gF_01_1221_1B.png', './pieces_dataV3/spectrum\\gF_01_1222_1B.png', './pieces_dataV3/spectrum\\gF_01_1223_1B.png', './pieces_dataV3/spectrum\\gF_01_1224_1B.png', './pieces_dataV3/spectrum\\gF_01_1225_1B.png', './pieces_dataV3/spectrum\\gF_01_1226_1B.png', './pieces_dataV3/spectrum\\gF_01_1227_1B.png', './pieces_dataV3/spectrum\\gF_01_1228_1B.png', './pieces_dataV3/spectrum\\gK_01_1629_1B.png', './pieces_dataV3/spectrum\\gK_01_1630_1B.png', './pieces_dataV3/spectrum\\gK_01_1631_1B.png', './pieces_dataV3/spectrum\\gK_01_1632_1B.png', './pieces_dataV3/spectrum\\gK_01_1633_1B.png', './pieces_dataV3/spectrum\\gM_01_1807_1B.png', './pieces_dataV3/spectrum\\gM_01_1808_1B.png', './pieces_dataV3/spectrum\\gM_01_1809_1B.png', './pieces_dataV3/spectrum\\gM_01_1810_1B.png', './pieces_dataV3/spectrum\\gM_01_1811_1B.png', './pieces_dataV3/spectrum\\gM_01_1812_1B.png', './pieces_dataV3/spectrum\\gM_01_1813_1B.png', './pieces_dataV3/spectrum\\gM_01_1814_1B.png', './pieces_dataV3/spectrum\\gP_01_2015_1C.png', './pieces_dataV3/spectrum\\gP_01_2016_1C.png', './pieces_dataV3/spectrum\\gP_01_2017_1C.png', './pieces_dataV3/spectrum\\gP_01_2018_1C.png', './pieces_dataV3/spectrum\\gP_01_2019_1C.png', './pieces_dataV3/spectrum\\gP_01_2020_1C.png', './pieces_dataV3/spectrum\\gP_01_2021_1C.png', './pieces_dataV3/spectrum\\gP_01_2022_1C.png', './pieces_dataV3/spectrum\\gP_01_2027_2B.png', './pieces_dataV3/spectrum\\gR_01_2269_1C.png', './pieces_dataV3/spectrum\\gR_01_2270_1C.png', './pieces_dataV3/spectrum\\gR_01_2271_1C.png', './pieces_dataV3/spectrum\\gR_01_2272_1C.png', './pieces_dataV3/spectrum\\gB_01_836_1C.png', './pieces_dataV3/spectrum\\gB_01_837_1C.png', './pieces_dataV3/spectrum\\gB_01_838_1C.png', './pieces_dataV3/spectrum\\gB_01_839_1C.png', './pieces_dataV3/spectrum\\gB_01_840_1C.png', './pieces_dataV3/spectrum\\gB_01_841_1C.png', './pieces_dataV3/spectrum\\gB_01_842_1C.png', './pieces_dataV3/spectrum\\gB_01_843_1C.png', './pieces_dataV3/spectrum\\gB_01_844_1C.png', './pieces_dataV3/spectrum\\gD_01_1035_1C.png', './pieces_dataV3/spectrum\\gD_01_1036_1C.png', './pieces_dataV3/spectrum\\gD_01_1037_1C.png', './pieces_dataV3/spectrum\\gD_01_1038_1C.png', './pieces_dataV3/spectrum\\gD_01_1039_1C.png', './pieces_dataV3/spectrum\\gD_01_1040_1C.png', './pieces_dataV3/spectrum\\gD_01_1041_1C.png', './pieces_dataV3/spectrum\\gD_01_1042_1C.png', './pieces_dataV3/spectrum\\gD_01_1043_1C.png', './pieces_dataV3/spectrum\\gD_01_1044_1C.png', './pieces_dataV3/spectrum\\gM_01_1816_1C.png', './pieces_dataV3/spectrum\\gM_01_1817_1C.png', './pieces_dataV3/spectrum\\gM_01_1818_1C.png', './pieces_dataV3/spectrum\\gM_01_1819_1C.png', './pieces_dataV3/spectrum\\gM_01_1820_1C.png', './pieces_dataV3/spectrum\\gM_01_1821_1C.png', './pieces_dataV3/spectrum\\gM_01_1822_1C.png', './pieces_dataV3/spectrum\\gM_01_1823_1C.png', './pieces_dataV3/spectrum\\gM_01_1824_1C.png', './pieces_dataV3/spectrum\\gP_01_2028_2B.png', './pieces_dataV3/spectrum\\gP_01_2029_2C.png', './pieces_dataV3/spectrum\\gP_01_2030_2C.png', './pieces_dataV3/spectrum\\gP_01_2031_2C.png', './pieces_dataV3/spectrum\\gP_01_2032_2C.png', './pieces_dataV3/spectrum\\gP_01_2033_2C.png', './pieces_dataV3/spectrum\\gQ_01_2162_2A.png', './pieces_dataV3/spectrum\\gQ_01_2163_2A.png', './pieces_dataV3/spectrum\\gQ_01_2164_2A.png', './pieces_dataV3/spectrum\\gQ_01_2165_2A.png', './pieces_dataV3/spectrum\\gQ_01_2166_2A.png', './pieces_dataV3/spectrum\\gR_01_2280_2C.png', './pieces_dataV3/spectrum\\gR_01_2281_2C.png', './pieces_dataV3/spectrum\\gR_01_2282_2C.png', './pieces_dataV3/spectrum\\gR_01_2283_2C.png', './pieces_dataV3/spectrum\\gR_01_2284_2C.png', './pieces_dataV3/spectrum\\gC_01_929_1C.png', './pieces_dataV3/spectrum\\gC_01_930_1C.png', './pieces_dataV3/spectrum\\gC_01_931_1C.png', './pieces_dataV3/spectrum\\gC_01_932_1C.png', './pieces_dataV3/spectrum\\gC_01_933_1C.png', './pieces_dataV3/spectrum\\gC_01_934_1C.png', './pieces_dataV3/spectrum\\gC_01_935_1C.png', './pieces_dataV3/spectrum\\gC_01_936_1C.png', './pieces_dataV3/spectrum\\gC_01_937_1C.png', './pieces_dataV3/spectrum\\gF_01_1241_2A.png', './pieces_dataV3/spectrum\\gF_01_1242_2A.png', './pieces_dataV3/spectrum\\gF_01_1243_2A.png', './pieces_dataV3/spectrum\\gF_01_1244_2A.png', './pieces_dataV3/spectrum\\gF_01_1245_2A.png', './pieces_dataV3/spectrum\\gF_01_1246_2A.png', './pieces_dataV3/spectrum\\gF_01_1247_2A.png', './pieces_dataV3/spectrum\\gF_01_1248_2A.png', './pieces_dataV3/spectrum\\gF_01_1249_2A.png', './pieces_dataV3/spectrum\\gF_01_1250_2A.png', './pieces_dataV3/spectrum\\gN_01_1929_2B.png', './pieces_dataV3/spectrum\\gN_01_1930_2B.png', './pieces_dataV3/spectrum\\gN_01_1931_2B.png', './pieces_dataV3/spectrum\\gN_01_1932_2B.png', './pieces_dataV3/spectrum\\gN_01_1933_2B.png', './pieces_dataV3/spectrum\\gN_01_1934_2B.png', './pieces_dataV3/spectrum\\gN_01_1935_2B.png', './pieces_dataV3/spectrum\\gN_01_1936_2B.png', './pieces_dataV3/spectrum\\gQ_01_2169_2B.png', './pieces_dataV3/spectrum\\gQ_01_2170_2B.png', './pieces_dataV3/spectrum\\gQ_01_2171_2B.png', './pieces_dataV3/spectrum\\gQ_01_2172_2B.png', './pieces_dataV3/spectrum\\gQ_01_2173_2C.png', './pieces_dataV3/spectrum\\gQ_01_2174_2C.png', './pieces_dataV3/spectrum\\gQ_01_2175_2C.png', './pieces_dataV3/spectrum\\gQ_01_2176_2C.png', './pieces_dataV3/spectrum\\gQ_01_2177_2C.png', './pieces_dataV3/spectrum\\gQ_01_2178_2C.png', './pieces_dataV3/spectrum\\gQ_01_2179_2C.png', './pieces_dataV3/spectrum\\gQ_01_2180_2C.png', './pieces_dataV3/spectrum\\gQ_01_2181_2C.png', './pieces_dataV3/spectrum\\gR_01_2292_3B.png', './pieces_dataV3/spectrum\\gR_01_2293_3B.png', './pieces_dataV3/spectrum\\gR_01_2294_3B.png', './pieces_dataV3/spectrum\\gR_01_2295_3B.png', './pieces_dataV3/spectrum\\gR_01_2296_3B.png', './pieces_dataV3/spectrum\\gR_01_2297_3C.png', './pieces_dataV3/spectrum\\gR_01_2298_3C.png', './pieces_dataV3/spectrum\\gR_01_2299_3C.png', './pieces_dataV3/spectrum\\gR_01_2300_3C.png', './pieces_dataV3/spectrum\\gR_01_2301_3C.png', './pieces_dataV3/spectrum\\gB_01_856_2B.png', './pieces_dataV3/spectrum\\gB_01_857_2B.png', './pieces_dataV3/spectrum\\gB_01_858_2B.png', './pieces_dataV3/spectrum\\gB_01_859_2B.png', './pieces_dataV3/spectrum\\gB_01_860_2B.png', './pieces_dataV3/spectrum\\gB_01_861_2B.png', './pieces_dataV3/spectrum\\gB_01_862_2B.png', './pieces_dataV3/spectrum\\gB_01_863_2B.png', './pieces_dataV3/spectrum\\gB_01_864_2B.png', './pieces_dataV3/spectrum\\gK_01_1656_2B.png', './pieces_dataV3/spectrum\\gK_01_1657_2B.png', './pieces_dataV3/spectrum\\gK_01_1658_2B.png', './pieces_dataV3/spectrum\\gK_01_1659_2B.png', './pieces_dataV3/spectrum\\gK_01_1660_2B.png', './pieces_dataV3/spectrum\\gK_01_1661_2B.png', './pieces_dataV3/spectrum\\gK_01_1662_2B.png', './pieces_dataV3/spectrum\\gK_01_1663_2B.png', './pieces_dataV3/spectrum\\gK_01_1664_2B.png', './pieces_dataV3/spectrum\\gL_01_1745_2A.png', './pieces_dataV3/spectrum\\gL_01_1746_2A.png', './pieces_dataV3/spectrum\\gL_01_1747_2A.png', './pieces_dataV3/spectrum\\gL_01_1748_2A.png', './pieces_dataV3/spectrum\\gN_01_1937_2C.png', './pieces_dataV3/spectrum\\gN_01_1938_2C.png', './pieces_dataV3/spectrum\\gN_01_1939_2C.png', './pieces_dataV3/spectrum\\gN_01_1940_2C.png', './pieces_dataV3/spectrum\\gN_01_1941_2C.png', './pieces_dataV3/spectrum\\gN_01_1942_2C.png', './pieces_dataV3/spectrum\\gN_01_1943_2C.png', './pieces_dataV3/spectrum\\gN_01_1944_2C.png', './pieces_dataV3/spectrum\\gQ_01_2183_3A.png', './pieces_dataV3/spectrum\\gQ_01_2184_3A.png', './pieces_dataV3/spectrum\\gQ_01_2185_3A.png', './pieces_dataV3/spectrum\\gQ_01_2186_3A.png', './pieces_dataV3/spectrum\\gQ_01_2187_3A.png', './pieces_dataV3/spectrum\\gQ_01_2188_3A.png', './pieces_dataV3/spectrum\\gQ_01_2189_3A.png', './pieces_dataV3/spectrum\\gQ_01_2190_3A.png', './pieces_dataV3/spectrum\\gR_01_2304_4A.png', './pieces_dataV3/spectrum\\gR_01_2305_4A.png', './pieces_dataV3/spectrum\\gR_01_2306_4A.png', './pieces_dataV3/spectrum\\gR_01_2307_4A.png', './pieces_dataV3/spectrum\\gR_01_2308_4A.png', './pieces_dataV3/spectrum\\gC_01_949_2B.png', './pieces_dataV3/spectrum\\gC_01_950_2B.png', './pieces_dataV3/spectrum\\gC_01_951_2B.png', './pieces_dataV3/spectrum\\gC_01_952_2B.png', './pieces_dataV3/spectrum\\gC_01_953_2B.png', './pieces_dataV3/spectrum\\gC_01_954_2B.png', './pieces_dataV3/spectrum\\gC_01_955_2B.png', './pieces_dataV3/spectrum\\gC_01_956_2B.png', './pieces_dataV3/spectrum\\gC_01_957_2B.png', './pieces_dataV3/spectrum\\gP_01_2067_3C.png', './pieces_dataV3/spectrum\\gP_01_2068_3C.png', './pieces_dataV3/spectrum\\gP_01_2069_3C.png', './pieces_dataV3/spectrum\\gP_01_2070_3C.png', './pieces_dataV3/spectrum\\gP_01_2071_3C.png', './pieces_dataV3/spectrum\\gP_01_2072_3C.png', './pieces_dataV3/spectrum\\gP_01_2073_3C.png', './pieces_dataV3/spectrum\\gP_01_2074_3C.png', './pieces_dataV3/spectrum\\gP_01_2075_3C.png', './pieces_dataV3/spectrum\\gQ_01_2201_3C.png', './pieces_dataV3/spectrum\\gQ_01_2202_3C.png', './pieces_dataV3/spectrum\\gQ_01_2203_3C.png', './pieces_dataV3/spectrum\\gQ_01_2204_3C.png', './pieces_dataV3/spectrum\\gQ_01_2205_3C.png', './pieces_dataV3/spectrum\\gQ_01_2206_3C.png', './pieces_dataV3/spectrum\\gR_01_2313_4C.png', './pieces_dataV3/spectrum\\gR_01_2314_4C.png', './pieces_dataV3/spectrum\\gR_01_2315_4C.png', './pieces_dataV3/spectrum\\gR_01_2316_4C.png', './pieces_dataV3/spectrum\\gR_01_2317_4C.png', './pieces_dataV3/spectrum\\gR_01_2318_4C.png', './pieces_dataV3/spectrum\\gG_01_1376_2C.png', './pieces_dataV3/spectrum\\gG_01_1377_2C.png', './pieces_dataV3/spectrum\\gG_01_1378_2C.png', './pieces_dataV3/spectrum\\gG_01_1379_2C.png', './pieces_dataV3/spectrum\\gG_01_1380_2C.png', './pieces_dataV3/spectrum\\gG_01_1381_2C.png', './pieces_dataV3/spectrum\\gG_01_1382_2C.png', './pieces_dataV3/spectrum\\gG_01_1383_2C.png', './pieces_dataV3/spectrum\\gG_01_1384_2C.png', './pieces_dataV3/spectrum\\gJ_01_1579_2A.png', './pieces_dataV3/spectrum\\gJ_01_1580_2A.png', './pieces_dataV3/spectrum\\gJ_01_1581_2A.png', './pieces_dataV3/spectrum\\gJ_01_1582_2A.png', './pieces_dataV3/spectrum\\gJ_01_1583_2A.png', './pieces_dataV3/spectrum\\gJ_01_1584_2A.png', './pieces_dataV3/spectrum\\gJ_01_1585_2A.png', './pieces_dataV3/spectrum\\gJ_01_1586_2A.png', './pieces_dataV3/spectrum\\gJ_01_1587_2A.png', './pieces_dataV3/spectrum\\gJ_01_1588_2A.png', './pieces_dataV3/spectrum\\gQ_01_2214_4C.png', './pieces_dataV3/spectrum\\gQ_01_2215_4C.png', './pieces_dataV3/spectrum\\gR_01_2327_5C.png', './pieces_dataV3/spectrum\\gR_01_2328_5C.png', './pieces_dataV3/spectrum\\gR_01_2329_5C.png', './pieces_dataV3/spectrum\\gR_01_2330_5C.png', './pieces_dataV3/spectrum\\gE_01_1183_3A.png', './pieces_dataV3/spectrum\\gE_01_1184_3A.png', './pieces_dataV3/spectrum\\gE_01_1185_3A.png', './pieces_dataV3/spectrum\\gE_01_1186_3A.png', './pieces_dataV3/spectrum\\gE_01_1187_3A.png', './pieces_dataV3/spectrum\\gE_01_1188_3A.png', './pieces_dataV3/spectrum\\gE_01_1189_3A.png', './pieces_dataV3/spectrum\\gE_01_1190_3A.png', './pieces_dataV3/spectrum\\gE_01_1191_3A.png', './pieces_dataV3/spectrum\\gF_01_1284_3A.png', './pieces_dataV3/spectrum\\gF_01_1285_3A.png', './pieces_dataV3/spectrum\\gF_01_1286_3A.png', './pieces_dataV3/spectrum\\gF_01_1287_3A.png', './pieces_dataV3/spectrum\\gF_01_1288_3A.png', './pieces_dataV3/spectrum\\gF_01_1289_3A.png', './pieces_dataV3/spectrum\\gF_01_1290_3A.png', './pieces_dataV3/spectrum\\gF_01_1291_3A.png', './pieces_dataV3/spectrum\\gF_01_1292_3A.png', './pieces_dataV3/spectrum\\gJ_01_1590_2B.png', './pieces_dataV3/spectrum\\gJ_01_1591_2B.png', './pieces_dataV3/spectrum\\gJ_01_1592_2B.png', './pieces_dataV3/spectrum\\gJ_01_1593_2B.png', './pieces_dataV3/spectrum\\gJ_01_1594_2B.png', './pieces_dataV3/spectrum\\gJ_01_1595_2B.png', './pieces_dataV3/spectrum\\gJ_01_1596_2B.png', './pieces_dataV3/spectrum\\gJ_01_1597_2B.png', './pieces_dataV3/spectrum\\gJ_01_1598_2B.png', './pieces_dataV3/spectrum\\gM_01_1867_3A.png', './pieces_dataV3/spectrum\\gM_01_1868_3A.png', './pieces_dataV3/spectrum\\gM_01_1869_3A.png', './pieces_dataV3/spectrum\\gM_01_1870_3A.png', './pieces_dataV3/spectrum\\gM_01_1871_3A.png', './pieces_dataV3/spectrum\\gM_01_1872_3A.png', './pieces_dataV3/spectrum\\gM_01_1873_3A.png', './pieces_dataV3/spectrum\\gN_01_1974_3C.png', './pieces_dataV3/spectrum\\gN_01_1975_3C.png', './pieces_dataV3/spectrum\\gN_01_1976_3C.png', './pieces_dataV3/spectrum\\gN_01_1977_3C.png', './pieces_dataV3/spectrum\\gN_01_1978_3C.png', './pieces_dataV3/spectrum\\gN_01_1979_3C.png', './pieces_dataV3/spectrum\\gN_01_1980_3C.png', './pieces_dataV3/spectrum\\gP_01_2098_4C.png', './pieces_dataV3/spectrum\\gP_01_2099_4C.png', './pieces_dataV3/spectrum\\gP_01_2100_4C.png', './pieces_dataV3/spectrum\\gP_01_2101_4C.png', './pieces_dataV3/spectrum\\gP_01_2102_4C.png', './pieces_dataV3/spectrum\\gP_01_2103_4C.png', './pieces_dataV3/spectrum\\gQ_01_2223_5B.png', './pieces_dataV3/spectrum\\gQ_01_2224_5B.png', './pieces_dataV3/spectrum\\gQ_01_2225_5B.png', './pieces_dataV3/spectrum\\gQ_01_2226_5C.png', './pieces_dataV3/spectrum\\gQ_01_2227_5C.png', './pieces_dataV3/spectrum\\gQ_01_2228_5C.png', './pieces_dataV3/spectrum\\gQ_01_2229_5C.png', './pieces_dataV3/spectrum\\gQ_01_2230_5C.png', './pieces_dataV3/spectrum\\gQ_01_2231_5C.png', './pieces_dataV3/spectrum\\gQ_01_2232_5C.png', './pieces_dataV3/spectrum\\gR_01_2338_6B.png', './pieces_dataV3/spectrum\\gR_01_2339_6B.png', './pieces_dataV3/spectrum\\gR_01_2340_6B.png', './pieces_dataV3/spectrum\\gR_01_2341_6B.png', './pieces_dataV3/spectrum\\gR_01_2342_6B.png', './pieces_dataV3/spectrum\\gR_01_2343_6B.png', './pieces_dataV3/spectrum\\gS_01_2434_3A.png', './pieces_dataV3/spectrum\\gS_01_2435_3A.png', './pieces_dataV3/spectrum\\gS_01_2436_3A.png', './pieces_dataV3/spectrum\\gS_01_2437_3A.png', './pieces_dataV3/spectrum\\gB_01_893_3B.png', './pieces_dataV3/spectrum\\gB_01_894_3B.png', './pieces_dataV3/spectrum\\gB_01_895_3B.png', './pieces_dataV3/spectrum\\gB_01_896_3B.png', './pieces_dataV3/spectrum\\gB_01_897_3B.png', './pieces_dataV3/spectrum\\gB_01_898_3B.png', './pieces_dataV3/spectrum\\gB_01_899_3B.png', './pieces_dataV3/spectrum\\gB_01_900_3B.png', './pieces_dataV3/spectrum\\gB_01_901_3B.png', './pieces_dataV3/spectrum\\gE_01_1192_3B.png', './pieces_dataV3/spectrum\\gE_01_1193_3B.png', './pieces_dataV3/spectrum\\gE_01_1194_3B.png', './pieces_dataV3/spectrum\\gE_01_1195_3B.png', './pieces_dataV3/spectrum\\gE_01_1196_3B.png', './pieces_dataV3/spectrum\\gE_01_1197_3B.png', './pieces_dataV3/spectrum\\gE_01_1198_3B.png', './pieces_dataV3/spectrum\\gE_01_1199_3B.png', './pieces_dataV3/spectrum\\gE_01_1200_3B.png', './pieces_dataV3/spectrum\\gF_01_1293_3B.png', './pieces_dataV3/spectrum\\gF_01_1294_3B.png', './pieces_dataV3/spectrum\\gF_01_1295_3B.png', './pieces_dataV3/spectrum\\gF_01_1296_3B.png', './pieces_dataV3/spectrum\\gF_01_1297_3B.png', './pieces_dataV3/spectrum\\gF_01_1298_3B.png', './pieces_dataV3/spectrum\\gF_01_1299_3B.png', './pieces_dataV3/spectrum\\gF_01_1300_3B.png', './pieces_dataV3/spectrum\\gF_01_1301_3B.png', './pieces_dataV3/spectrum\\gF_01_1302_3B.png', './pieces_dataV3/spectrum\\gL_01_1779_3B.png', './pieces_dataV3/spectrum\\gL_01_1780_3B.png', './pieces_dataV3/spectrum\\gL_01_1781_3B.png', './pieces_dataV3/spectrum\\gL_01_1782_3B.png', './pieces_dataV3/spectrum\\gL_01_1783_3B.png', './pieces_dataV3/spectrum\\gL_01_1784_3B.png', './pieces_dataV3/spectrum\\gL_01_1785_3B.png', './pieces_dataV3/spectrum\\gL_01_1786_3B.png', './pieces_dataV3/spectrum\\gN_01_1982_4A.png', './pieces_dataV3/spectrum\\gN_01_1983_4A.png', './pieces_dataV3/spectrum\\gN_01_1984_4A.png', './pieces_dataV3/spectrum\\gN_01_1985_4A.png', './pieces_dataV3/spectrum\\gN_01_1986_4A.png', './pieces_dataV3/spectrum\\gN_01_1987_4A.png', './pieces_dataV3/spectrum\\gN_01_1988_4A.png', './pieces_dataV3/spectrum\\gP_01_2105_5A.png', './pieces_dataV3/spectrum\\gP_01_2106_5A.png', './pieces_dataV3/spectrum\\gP_01_2107_5A.png', './pieces_dataV3/spectrum\\gP_01_2108_5A.png', './pieces_dataV3/spectrum\\gP_01_2109_5A.png', './pieces_dataV3/spectrum\\gP_01_2110_5A.png', './pieces_dataV3/spectrum\\gQ_01_2235_6A.png', './pieces_dataV3/spectrum\\gQ_01_2236_6A.png', './pieces_dataV3/spectrum\\gQ_01_2237_6A.png', './pieces_dataV3/spectrum\\gQ_01_2238_6A.png', './pieces_dataV3/spectrum\\gQ_01_2239_6A.png', './pieces_dataV3/spectrum\\gQ_01_2240_6A.png', './pieces_dataV3/spectrum\\gQ_01_2241_6A.png', './pieces_dataV3/spectrum\\gQ_01_2242_6A.png', './pieces_dataV3/spectrum\\gQ_01_2243_6A.png', './pieces_dataV3/spectrum\\gQ_01_2244_6A.png', './pieces_dataV3/spectrum\\gR_01_2346_6C.png', './pieces_dataV3/spectrum\\gR_01_2347_6C.png', './pieces_dataV3/spectrum\\gR_01_2348_6C.png', './pieces_dataV3/spectrum\\gS_01_2439_3B.png', './pieces_dataV3/spectrum\\gS_01_2440_3B.png', './pieces_dataV3/spectrum\\gS_01_2441_3B.png', './pieces_dataV3/spectrum\\gS_01_2442_3B.png', './pieces_dataV3/spectrum\\gS_01_2443_3B.png', './pieces_dataV3/spectrum\\gS_01_2444_3B.png', './pieces_dataV3/spectrum\\gB_01_903_3C.png', './pieces_dataV3/spectrum\\gB_01_904_3C.png', './pieces_dataV3/spectrum\\gB_01_905_3C.png', './pieces_dataV3/spectrum\\gB_01_906_3C.png', './pieces_dataV3/spectrum\\gB_01_907_3C.png', './pieces_dataV3/spectrum\\gB_01_908_3C.png', './pieces_dataV3/spectrum\\gB_01_909_3C.png', './pieces_dataV3/spectrum\\gB_01_910_3C.png', './pieces_dataV3/spectrum\\gB_01_911_3C.png', './pieces_dataV3/spectrum\\gB_01_912_3C.png', './pieces_dataV3/spectrum\\gE_01_1202_3C.png', './pieces_dataV3/spectrum\\gE_01_1203_3C.png', './pieces_dataV3/spectrum\\gE_01_1204_3C.png', './pieces_dataV3/spectrum\\gE_01_1205_3C.png', './pieces_dataV3/spectrum\\gE_01_1206_3C.png', './pieces_dataV3/spectrum\\gE_01_1207_3C.png', './pieces_dataV3/spectrum\\gE_01_1208_3C.png', './pieces_dataV3/spectrum\\gE_01_1209_3C.png', './pieces_dataV3/spectrum\\gE_01_1210_3C.png', './pieces_dataV3/spectrum\\gF_01_1304_3C.png', './pieces_dataV3/spectrum\\gF_01_1305_3C.png', './pieces_dataV3/spectrum\\gF_01_1306_3C.png', './pieces_dataV3/spectrum\\gF_01_1307_3C.png', './pieces_dataV3/spectrum\\gF_01_1308_3C.png', './pieces_dataV3/spectrum\\gF_01_1309_3C.png', './pieces_dataV3/spectrum\\gF_01_1310_3C.png', './pieces_dataV3/spectrum\\gF_01_1311_3C.png', './pieces_dataV3/spectrum\\gF_01_1312_3C.png', './pieces_dataV3/spectrum\\gF_01_1313_3C.png', './pieces_dataV3/spectrum\\gL_01_1787_3C.png', './pieces_dataV3/spectrum\\gL_01_1788_3C.png', './pieces_dataV3/spectrum\\gL_01_1789_3C.png', './pieces_dataV3/spectrum\\gL_01_1790_3C.png', './pieces_dataV3/spectrum\\gL_01_1791_3C.png', './pieces_dataV3/spectrum\\gL_01_1792_3C.png', './pieces_dataV3/spectrum\\gL_01_1793_3C.png', './pieces_dataV3/spectrum\\gL_01_1794_3C.png', './pieces_dataV3/spectrum\\gL_01_1795_3C.png', './pieces_dataV3/spectrum\\gN_01_1996_4C.png', './pieces_dataV3/spectrum\\gN_01_1997_4C.png', './pieces_dataV3/spectrum\\gN_01_1998_4C.png', './pieces_dataV3/spectrum\\gN_01_1999_4C.png', './pieces_dataV3/spectrum\\gN_01_2000_4C.png', './pieces_dataV3/spectrum\\gN_01_2001_4C.png', './pieces_dataV3/spectrum\\gN_01_2002_4C.png', './pieces_dataV3/spectrum\\gP_01_2121_5C.png', './pieces_dataV3/spectrum\\gP_01_2122_5C.png', './pieces_dataV3/spectrum\\gP_01_2123_5C.png', './pieces_dataV3/spectrum\\gP_01_2124_5C.png', './pieces_dataV3/spectrum\\gP_01_2125_5C.png', './pieces_dataV3/spectrum\\gP_01_2126_5C.png', './pieces_dataV3/spectrum\\gP_01_2127_5C.png', './pieces_dataV3/spectrum\\gP_01_2128_5C.png', './pieces_dataV3/spectrum\\gP_01_2129_5C.png', './pieces_dataV3/spectrum\\gP_01_2130_5C.png', './pieces_dataV3/spectrum\\gQ_01_2254_6C.png', './pieces_dataV3/spectrum\\gQ_01_2255_6C.png', './pieces_dataV3/spectrum\\gQ_01_2256_6C.png', './pieces_dataV3/spectrum\\gQ_01_2257_6C.png', './pieces_dataV3/spectrum\\gQ_01_2258_6C.png', './pieces_dataV3/spectrum\\gR_01_2362_8A.png', './pieces_dataV3/spectrum\\gR_01_2363_8A.png', './pieces_dataV3/spectrum\\gS_01_2447_3C.png', './pieces_dataV3/spectrum\\gS_01_2448_3C.png', './pieces_dataV3/spectrum\\gS_01_2449_3C.png', './pieces_dataV3/spectrum\\gS_01_2450_3C.png', './pieces_dataV3/spectrum\\gS_01_2451_3C.png', './pieces_dataV3/spectrum\\gS_01_2452_3C.png', './pieces_dataV3/spectrum\\gS_01_2453_3C.png', './pieces_dataV3/spectrum\\gS_01_2454_3C.png', './pieces_dataV3/spectrum\\gS_01_2455_3C.png']]

for i in range(numClassifiers):
    count = [0,0,0]
    relativeFrequency = [0,0,0]
    print(classifierNames[i])
    for el in wrongFiles[i]:
        side = el[-5]
        if side == "A":
            count[0] += 1
        elif side == "B":
            count[1] += 1
        else:
            count[2] += 1
        total = count[0]+count[1]+count[2]
    for j in range(len(count)):
        relativeFrequency[j]=count[j]/total
    print("Relative frequency of wrongly predicted sides (A, B, C): ",relativeFrequency, "\n\n")